# Imports and connect to drive, reproduce settings

In [ ]:
import pandas as pd
import pandas_datareader as web
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torch.utils.data as data_utils
import time
import random
import string
import numpy

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from torchtext.vocab import build_vocab_from_iterator
import torchtext

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score 
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer

import pickle

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install --upgrade pandas
!pip install --upgrade pandas-datareader

In [ ]:
pip install transformers

In [ ]:
from transformers import BertTokenizer
from transformers import BertTokenizer, BertModel

bert = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# reproduce
seed = 1

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
random.seed(seed)

In [ ]:
# todo define device here and remove the device def from after
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
pip install vaderSentiment

  Using cached vaderSentiment-3.3.2-py2.py3-none-any.whl (125 kB)


In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Prepare the numerical stock data

In [ ]:
# Copy the dataset to the local environment
#!cp "/content/drive/MyDrive/upload_DJIA_table.csv" "DJIA.csv"

# Load the stock data
#change back to kaggle dataset, however I know it has some problem in it
#df_stock = pd.read_csv('DJIA.csv', index_col = "Date")

# Load the stock data
df_stock = web.DataReader("DJIA", data_source="yahoo", start="2008-08-08", 
                          end="2016-07-01")
df_stock.sort_index(axis=0,ascending=True,inplace=True)

#check if close and adj close is ever different
diff_index = []
for row in range(len(df_stock)):
    if df_stock["Close"][row] != df_stock["Adj Close"][row]:
        diff_index.append(row)

if len(diff_index) > 1:
  raise ValueError("DIFFERENT")

# Calculate moving average for 7, 14, 21 days
ma_day = [7, 14, 21]

for ma in ma_day:
  column_name = f"MA for {ma} days"
  df_stock[column_name] = df_stock["Adj Close"].rolling(ma).mean()  

#calculate the diff volume for the before day
diff_volume = []
for row in range(len(df_stock)):
    if row == 0:
        diff_volume.append(0)
    else:
        diff_volume.append(df_stock["Volume"][row] - df_stock["Volume"][row - 1])

df_stock["Volume diff"] = diff_volume

for ma in ma_day:
  column_name = f"Volume diff MA for {ma} days"
  df_stock[column_name] = df_stock["Volume diff"].rolling(ma).mean()
  break # only use 7 MA

#H-L with MA
diff_H_L = []
for row in range(len(df_stock)):
    diff_H_L.append(df_stock["High"][row] - df_stock["Low"][row])

df_stock["High-Low"] = diff_H_L

#O-C with MA
diff_O_C = []
for row in range(len(df_stock)):
    diff_O_C.append(df_stock["Open"][row] - df_stock["Close"][row])

df_stock["Open-Close"] = diff_O_C

#close standard deviation
df_stock["Close 7day deviation"] = df_stock["Close"].rolling(7).std()

#create input and output data
input_df = pd.DataFrame()
output_df = pd.DataFrame()

#input features:
#   Open
#   High
#   Low
#   Close
#   Volume
#   High - Low
#   Open - Close
#   7 days MA close
#   14 days MA close
#   21 days MA close
#   7 days MA volume
#   7 days std 
input_df["Open"] = df_stock["Open"][21:]
input_df["High"] = df_stock["Open"][21:]
input_df["Low"] = df_stock["Low"][21:]
input_df["Close"] = df_stock["Close"][21:]
input_df["Volume"] = df_stock["Volume"][21:]
input_df["High-Low"] = df_stock["High-Low"][21:]
input_df["Open-Close"] = df_stock["Open-Close"][21:]
input_df["7d MA close"] = df_stock["MA for 7 days"][21:]
input_df["14d MA close"] = df_stock["MA for 14 days"][21:]
input_df["21d MA close"] = df_stock["MA for 21 days"][21:]
input_df["7d MA volume"] = df_stock["Volume diff MA for 7 days"][21:]
input_df["14d std"] = df_stock["Close 7day deviation"][21:]

#output:
#   close for next day
#input start from 24. day and last to last before one,
#output start from 25. day and last to the last
#will drop the rows after std for get the same values
output_df["Next Close"] = df_stock["Close"][21:]

# std the input and output
input_std = (input_df - input_df.mean())/input_df.std()

output_std = (output_df - output_df.mean())/output_df.std()

# drop the input last row
input_std.drop(input_std.tail(1).index,inplace=True)

# drop the output first row
output_std.drop(output_std.head(1).index,inplace=True)
###################################### from here

data_num = input_std
data_num["Next Close"] = output_std["Next Close"].values

# Prepare the stock news information

In [ ]:
# Copy the dataset to the local environment
!cp "/content/drive/MyDrive/Combined_News_DJIA.csv" "Combined_News_DJIA.csv"

# Load the dataset 
df_combined = pd.read_csv('Combined_News_DJIA.csv', index_col = "Date")

# Find the cells with NaN and after the rows for them
is_NaN = df_combined.isnull()
row_has_NaN = is_NaN.any(axis = 1)
rows_with_NaN = df_combined[row_has_NaN]

# Replace them
df_combined = df_combined.replace(np.nan, " ")

# Check the process
is_NaN = df_combined.isnull()
row_has_NaN = is_NaN.any(axis = 1)
rows_with_NaN = df_combined[row_has_NaN]

if len(rows_with_NaN) != 0:
    raise ValueError("There is NaN in news dataset!")

# Drop the Label and change it with trend target
df_combined.drop(["Label"], axis=1,inplace=True)

# Get column names
combined_column_names = []
for column in df_combined.columns:
  combined_column_names.append(column)

# 2D array creation for the news based on macros
COLUMNS = len(df_combined)
ROWS = 25
news_sum = [[0 for i in range(COLUMNS)] for j in range(int(len(combined_column_names) / ROWS))]

# Merge the news
for row in range(len(df_combined)):
  for column in range(int(len(combined_column_names) / ROWS)):
    temp = ""
    news = ""
    for word in range(ROWS):
      news = df_combined[combined_column_names[(column * ROWS) + word]][row]
      # Remove the b character at the begining of the string
      if news[0] is "b":
        news = " " + news[1:]
      temp = temp + news
    news_sum[column][row] = temp

# Removing punctuations and numbers
# Also removes the stop words
# using tokenizer
stop_words = set(stopwords.words('english'))
for line_i in range(len(news_sum)):
  for row_i in range(len(news_sum[line_i])):
    word_tokens = word_tokenize(news_sum[line_i][row_i])
    temp_attach = ""
    for word in word_tokens:
      temp = ""
      word = word.translate(str.maketrans('', '', string.punctuation)) # remove punctations
      if word.isdigit() is False: # remove number
        if word not in stop_words: # remove stop words
          temp = word.lower() # converting lower case
      temp_attach = temp_attach + " " + temp # add the word
    temp_attach = " ".join(temp_attach.split()) # remove space
    news_sum[line_i][row_i] = temp_attach

# Drop the old columns
for column in range(len(combined_column_names)):
  df_combined.drop(combined_column_names[column], axis = 1, inplace = True)

# Create the new columns with the merged news
for column in range(int(len(combined_column_names) / ROWS)):
  colum_name = "News_" + str(column + 1)
  df_combined[colum_name] = news_sum[column]      

# Prepare the complex dataset

In [ ]:
# join the two dataset and create the trend target for the news dataset
data_num.index = data_num.index.strftime('%Y-%m-%d')
complex_dataset = data_num.join(df_combined)

#generate the trend target (shows the next day)
#1 -> rise, otherwise 0
#the model is the today's news how to change the next day trend !
trend_target = []
for element in range(len(complex_dataset)):
  if complex_dataset["Close"][element] > complex_dataset["Next Close"][element]:
      trend_target.append(0)
  else:
      trend_target.append(1)  

complex_dataset["Trend target"] = trend_target

# Check the process
is_NaN = complex_dataset.isnull()
row_has_NaN = is_NaN.any(axis = 1)
rows_with_NaN = complex_dataset[row_has_NaN]

if len(rows_with_NaN) != 0:
    raise ValueError("There is NaN in news dataset!")

In [ ]:
# output for the complex model is the next close
#split to 80% train+valid and 20% test
train_valid, test = np.split(complex_dataset, [int(.8*len(complex_dataset))])

#shuffle and split 60% 20% the train and valid
#80% is train+valid -> 75% of train+valid is 60% train from the complete
train, valid = np.split(train_valid, [int(.75*len(train_valid))])

numeric_columns = []
for i in range(complex_dataset.columns.get_loc("Next Close")+1):
  numeric_columns.append(i)
train_num = train.iloc[:, numeric_columns]
valid_num = valid.iloc[:, numeric_columns]
test_num = test.iloc[:, numeric_columns]

news_columns = []
for i in range(complex_dataset.columns.get_loc("Next Close") + 1,len(complex_dataset.columns)):
  news_columns.append(i)
train_news = train.iloc[:, news_columns]
valid_news = valid.iloc[:, news_columns]
test_news = test.iloc[:, news_columns]

train_Y_num = torch.tensor(train_num["Next Close"].values.astype(np.float32))
train_Y_num = train_Y_num.view(-1,1)
train_X_num = torch.tensor(train_num.drop(["Next Close"],axis=1).values.astype(np.float32))

valid_Y_num = torch.tensor(valid_num["Next Close"].values.astype(np.float32))
valid_Y_num = valid_Y_num.view(-1,1)
valid_X_num = torch.tensor(valid_num.drop("Next Close",axis=1).values.astype(np.float32))

test_Y_num = torch.tensor(test_num["Next Close"].values.astype(np.float32))
test_Y_num = test_Y_num.view(-1,1)
test_X_num = torch.tensor(test_num.drop("Next Close",axis=1).values.astype(np.float32))

train_Y_news = torch.tensor(train_news["Trend target"].values.astype(np.float32))
train_Y_news = train_Y_news.view(-1,1)
train_X_news_raw = train_news.drop("Trend target",axis=1)

valid_Y_news = torch.tensor(valid_news["Trend target"].values.astype(np.float32))
valid_Y_news = valid_Y_news.view(-1,1)
valid_X_news_raw = valid_news.drop("Trend target",axis=1)

test_Y_news = torch.tensor(test_news["Trend target"].values.astype(np.float32))
test_Y_news = test_Y_news.view(-1,1)
test_X_news_raw = test_news.drop("Trend target",axis=1)

train_Y_all = torch.tensor(train["Next Close"].values.astype(np.float32))
train_Y_all = train_Y_all.view(-1,1)
train_X_all_raw = train.drop(["Next Close","Trend target"],axis=1)

valid_Y_all = torch.tensor(valid["Next Close"].values.astype(np.float32))
valid_Y_all = valid_Y_all.view(-1,1)
valid_X_all_raw = valid.drop(["Next Close","Trend target"],axis=1)

test_Y_all = torch.tensor(test["Next Close"].values.astype(np.float32))
test_Y_all = test_Y_all.view(-1,1)
test_X_all_raw = test.drop(["Next Close","Trend target"],axis=1)

In [ ]:
def preprocess_raw_text(tokenizer):
    global vocab
    if tokenizer == "simple":
        def yield_tokens(data_iter):
            for text in data_iter:
                yield word_tokenize(text)

        min_freq = 30
        special_tokens = ["<unk>", "<pad>"]        

        vocab = build_vocab_from_iterator(yield_tokens(train_X_news_raw["News_1"]),
                                          min_freq = min_freq,
                                          specials=special_tokens)

        global pretrained_embedding
        # uncomment if no saved pretrained
        #pretrained_vectors = torchtext.vocab.GloVe(name='6B', dim=50)
        #pretrained_embedding = pretrained_vectors.get_vecs_by_tokens(vocab.get_itos())
        # save out the pretrained embedding
        #torch.save(pretrained_embedding, "drive/MyDrive/complex/pretrained_embeddings.pt")

        # load the pretrained embeddings
        pretrained_embedding = torch.load("drive/MyDrive/complex/pretrained_embeddings.pt")
        # can model.embedding.weight.data = pretrained_embedding

        # set deafult vocab token
        vocab.set_default_index(vocab.lookup_indices(["<unk>"])[0])

        # setting padding id
        pad_id = vocab.lookup_indices(["<pad>"])[0]

        text_pipeline = lambda x: vocab(word_tokenize(x))

    elif tokenizer == "bert":
      tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

      max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']

      def text_pipeline(text):
          tokens = tokenizer.tokenize(text)
          if len(tokens) > max_input_length-2:
              print(f"Text is bigger than the max length: {len(tokens)}")
          tokens = tokens[:max_input_length-2]
          ids = tokenizer.convert_tokens_to_ids(tokens)
          return ids     

      pad_id = tokenizer.pad_token_id 

      longest_new = max_input_length-2

    elif tokenizer == "VADER":
      analyzer = SentimentIntensityAnalyzer()

      longest_new = 1

      # output is raw VADER sentiment
      def text_pipeline(text):
        vs = analyzer.polarity_scores(text)
        retVal = []
        retVal.append(vs.get("compound"))
        return retVal       

      pad_id = 0 

    elif tokenizer == "logreg": # create vocab with every word, then after batch reconvert it 
        def yield_tokens(data_iter):
            for text in data_iter:
                yield word_tokenize(text)

        min_freq = 0
        special_tokens = ["<unk>", "<pad>"]        

        vocab = build_vocab_from_iterator(yield_tokens(train_X_news_raw["News_1"]),
                                          min_freq = min_freq,
                                          specials=special_tokens)

        # set deafult vocab token
        vocab.set_default_index(vocab.lookup_indices(["<unk>"])[0])

        # setting padding id
        pad_id = vocab.lookup_indices(["<pad>"])[0]

        text_pipeline = lambda x: vocab(word_tokenize(x)) 

        bow_vectorizer = CountVectorizer()
        bow_train = bow_vectorizer.fit_transform(train_X_news_raw["News_1"])     

    else:
        raise ValueError("No suitable tokenizer given!")     

    # tokenize all of the news and create the tokens from vocab
    train_news = []
    valid_news = []
    test_news = []
    for new in train_X_news_raw["News_1"]:
        processed_text = text_pipeline(new)
        train_news.append(processed_text)
    for new in valid_X_news_raw["News_1"]:
        processed_text = text_pipeline(new)
        valid_news.append(processed_text)
    for new in test_X_news_raw["News_1"]:
        processed_text = text_pipeline(new)
        test_news.append(processed_text)   

    if tokenizer == "simple" or tokenizer == "logreg":
        # search for the longest
        longest_new = 0
        for new in train_news:
            if len(new) > longest_new:
              longest_new = len(new)
        for new in valid_news:
            if len(new) > longest_new:
              longest_new = len(new)
        for new in test_news:
            if len(new) > longest_new:
              longest_new = len(new)       

    # pad the news tokens to the same length
    # do the pad
    train_news_temp = []
    valid_news_temp = []
    test_news_temp = []
    for new in train_news:
        train_news_temp.append(
            numpy.pad(new,
                      (0,
                      longest_new-len(new)),
                      mode="constant",
                      constant_values=pad_id))
    for new in valid_news:
        valid_news_temp.append(
            numpy.pad(new,
                      (0,
                      longest_new-len(new)),
                      mode="constant",
                      constant_values=pad_id))
    for new in test_news:
        test_news_temp.append(
            numpy.pad(new,
                      (0,
                      longest_new-len(new)),
                      mode="constant",
                      constant_values=pad_id))
        
    if tokenizer == "VADER":
      train_X_news = torch.FloatTensor(train_news_temp)
      valid_X_news = torch.FloatTensor(valid_news_temp)
      test_X_news = torch.FloatTensor(test_news_temp)  
    else:  
      train_X_news = torch.LongTensor(train_news_temp)
      valid_X_news = torch.LongTensor(valid_news_temp)
      test_X_news = torch.LongTensor(test_news_temp)    

    return train_X_news, valid_X_news, test_X_news

In [ ]:
# preprocess raw text
train_X_news, valid_X_news, test_X_news = preprocess_raw_text("bert")

# create tensor datasets and loaders
# only numerical data
train_num_tensor = data_utils.TensorDataset(train_X_num, train_Y_num)
valid_num_tensor = data_utils.TensorDataset(valid_X_num, valid_Y_num)
test_num_tensor = data_utils.TensorDataset(test_X_num, test_Y_num)

train_num_loader = data_utils.DataLoader(dataset = train_num_tensor, 
                              batch_size = 32, shuffle = False)
valid_num_loader = data_utils.DataLoader(dataset = valid_num_tensor, 
                              batch_size = 32, shuffle = False)  
test_num_loader = data_utils.DataLoader(dataset = test_num_tensor, 
                              batch_size = 32, shuffle = False)

# news and labels data only
train_news_tensor = data_utils.TensorDataset(train_X_news, train_Y_news)
valid_news_tensor = data_utils.TensorDataset(valid_X_news, valid_Y_news)
test_news_tensor = data_utils.TensorDataset(test_X_news, test_Y_news)

train_news_loader = data_utils.DataLoader(dataset = train_news_tensor, 
                              batch_size = 32, shuffle = False)
valid_news_loader = data_utils.DataLoader(dataset = valid_news_tensor, 
                              batch_size = 32, shuffle = False)  
test_news_loader = data_utils.DataLoader(dataset = test_news_tensor, 
                              batch_size = 32, shuffle = False)

# all data (news + numerical)
# concat the numeric and news tensor dataset
train_all_tensor = data_utils.TensorDataset(train_X_num, train_X_news,  train_Y_num)
valid_all_tensor = data_utils.TensorDataset(valid_X_num, valid_X_news,  valid_Y_num)
test_all_tensor = data_utils.TensorDataset(test_X_num, test_X_news,  test_Y_num)

train_all_loader = data_utils.DataLoader(dataset = train_all_tensor, 
                              batch_size = 32, shuffle = False)
valid_all_loader = data_utils.DataLoader(dataset = valid_all_tensor, 
                              batch_size = 32, shuffle = False)  
test_all_loader = data_utils.DataLoader(dataset = test_all_tensor, 
                              batch_size = 32, shuffle = False)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Text is bigger than the max length: 519
Text is bigger than the max length: 522
Text is bigger than the max length: 529
Text is bigger than the max length: 562
Text is bigger than the max length: 514
Text is bigger than the max length: 584
Text is bigger than the max length: 526


# Set up the models

## Numerical

In [ ]:
# set up a numerical model
# linear 108
# LR start/end 0.0001, batch 32, dropout 0, layer d 11, activition none, score 70.63 were 
# but needed retrain for not shuffled dataset -> to be coherent with news dataset
class Linear_108_num(nn.Module):
    def __init__(self): # 
        super(Linear_108_num, self).__init__()

        self.fc = nn.ModuleList()
        self.fc.append(nn.Linear(12, 64))
        self.fc.append(nn.Linear(64, 128))
        self.fc.append(nn.Linear(128, 256))
        self.fc.append(nn.Linear(256, 512))
        self.fc.append(nn.Linear(512, 1024))  
        self.fc.append(nn.Linear(1024, 512))               
        self.fc.append(nn.Linear(512, 256))
        self.fc.append(nn.Linear(256, 128))
        self.fc.append(nn.Linear(128, 64))
        self.fc.append(nn.Linear(64, 16))
        self.fc.append(nn.Linear(16, 1))

    def forward(self, x):
        for fc in self.fc:
            x = fc(x)
        return x        

# LSTM 92 -> these names not OK, they werent suitable for the new dataset split...
# batch size 32, LR start and end 0.001 -> no dynamic, dropout 0.1, architect 223,
# activition function none, bidirect false; score 83.5 and acc was 48.35
class LSTM_92_num(nn.Module):
    def __init__(self):
        super(LSTM_92_num, self).__init__()
  
        self.LSTM = nn.LSTM(input_size=12,
                                hidden_size=256,
                                num_layers=2,
                                bidirectional=True,
                                dropout=0.1)
        self.fc1 = nn.Linear(2*256,64)
        self.fc2 = nn.Linear(64,8)   
        self.fc3 = nn.Linear(8,1)   

        self.dropout = nn.Dropout(0.1)    
        self.batch_size = 32                                                                                                                                          

    def forward(self, x):      
        try:
          x_reshaped = torch.reshape(x,(-1,32,12))
        except:
          x_reshaped = torch.reshape(x,(-1,1,12))

        output, (hidden, cell) = self.LSTM(x_reshaped)

        x = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)

        return x

# LSTM 36
# batch size 32, LR start and end 0.001 -> no dynamic, dropout 0, architect 321,
# activition function none, bidirect false; score 117.546 and acc was 52.42
class LSTM_36_num(nn.Module):
    def __init__(self):
        super(LSTM_36_num, self).__init__()
  
        self.LSTM = nn.LSTM(input_size=12,
                                hidden_size=128,
                                num_layers=2,
                                bidirectional=True,
                                dropout=0.1)
        self.fc1 = nn.Linear(2*128,16)
        self.fc2 = nn.Linear(16,1)   

        self.dropout = nn.Dropout(0.1)    
        self.batch_size = 32                                                                                                                                          

    def forward(self, x):      
        try:
          x_reshaped = torch.reshape(x,(-1,32,12))
        except:
          x_reshaped = torch.reshape(x,(-1,1,12))

        output, (hidden, cell) = self.LSTM(x_reshaped)

        x = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        x = self.fc1(x)
        x = self.fc2(x)

        return x

## News

In [ ]:
# set up a NLP model
# LSTM
# merged news into 1, embedding dims 300, hidden size 32, layer 2, 3 numerical after
# biderictional yes, dropout 0.4, ReLU act, sigmoid activiton at the end
class LSTM_news(nn.Module):
    def __init__(self, vocab_size, pad_idx):        
        super(LSTM_news, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, 50, padding_idx = pad_idx)  

        self.rnn = nn.LSTM(50, #input size
                          32, #features in hidden state
                          num_layers=2, 
                          bidirectional=True, 
                          dropout=0.4)        
       
        self.fc1 = nn.Linear(2*32, 32)
        self.fc2 = nn.Linear(32, 8)
        self.fc3 = nn.Linear(8, 1)

        self.activition = nn.ReLU()
        self.dropout = nn.Dropout(0.4)
        
    def forward(self, text):
        try:
          text = torch.reshape(text,(464,32))
        except:
          text = torch.reshape(text,(464,1))

        embedded = self.embedding(text)

        output, (hidden, cell) = self.rnn(embedded)
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        #this is for Bidirectional only!
        hidden = self.dropout(self.activition(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))

        x = self.activition(self.fc1(hidden))
        x = self.activition(self.fc2(x))
        x = self.fc3(x)

        sig = nn.Sigmoid()
        x = sig(x)  

        return x

# Bert
class BERT_news(nn.Module):
    def __init__(self):        
        super(BERT_news, self).__init__()
        
        self.bert = bert.to(device)
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.rnn = nn.GRU(input_size=embedding_dim,
                          hidden_size=32,
                          num_layers=2,
                          bidirectional=False,
                          batch_first = True,
                          dropout = 0.3)
        
        self.out = nn.Linear(32, 1)
        
        self.activition = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        
    def forward(self, text):                
        with torch.no_grad():
            embedded = self.bert(text)[0]
        
        _, hidden = self.rnn(embedded)
        
        hidden = self.activition(self.dropout(hidden[-1,:,:]))
        
        sig = nn.Sigmoid()
        output = sig(self.out(hidden))   
        return output       

# VADER
class VADER():
    def __init__(self):        
        super(VADER, self).__init__()
        
    def forward(self, text):                        
        sig = nn.Sigmoid()
        output = sig(-1*text)
        return output

# logreg
class LogReg():
    def __init__(self):        
        super(LogReg, self).__init__()

    def convertIndicesToToken(self, news, train):
      if train == True:
        tokens = []
        for new in news:
          token_row = ""
          for indice in new:
              token = vocab.lookup_token(indice)
              if token != "<unk>" and token != "<pad>":
                  token_row += token + " "
          tokens.append(token_row)   
      else:
        tokens_tmp = ""
        for indice in news:
            token = vocab.lookup_token(indice)
            if token != "<unk>" and token != "<pad>":
                tokens_tmp += token + " "
        tokens = []
        tokens.append(tokens_tmp)

      return tokens        

    def train(self, train_x, train_y):
        # reconvert indices to tokens
        train_x = self.convertIndicesToToken(train_x, train=True)

        # fit vectorizer
        bow_vectorizer = CountVectorizer()
        bow_vectorizer.fit(train_x)

        # fit model
        bow_train = bow_vectorizer.transform(train_x)
        bow_model = LogisticRegression(random_state=seed, max_iter=100000,solver='lbfgs')
        bow_model = bow_model.fit(bow_train, torch.reshape(train_y,(-1,)))

        self.bow_vectorizer = bow_vectorizer
        self.bow_model = bow_model

        print("Train done")

    def forward(self, text):      
        # reconvert indices to tokens
        try:
          text = self.convertIndicesToToken(text, train=False)
        except:
          text = self.convertIndicesToToken(text, train=True)

        bow_test = self.bow_vectorizer.transform(text)
        bow_prediction = self.bow_model.predict_proba(bow_test)
        prob_out = []
        for predict_prob in bow_prediction:
            prob_out.append(predict_prob[1])

        return torch.reshape(torch.FloatTensor(prob_out),(len(prob_out),1))                  

## Complex

In [ ]:
# complex model 1
# 4 layer linear multi layer after
class Complex_1(nn.Module):
    def __init__(self, model_num, model_news):  
        super(Complex_1, self).__init__()

        self.model_num = model_num
        self.model_news = model_news

        # create the NN after the two outputs
        # input -> trend probability from news and std. close prediction
        # output -> close prediction
        self.fc1 = nn.Linear(2,8)
        self.fc2 = nn.Linear(8,32)
        self.fc3 = nn.Linear(32,16)
        self.fc4 = nn.Linear(16,1)

    def forward(self, x_num, x_news):
        global y_num,y_news
        y_num = self.model_num(x_num)
        try:
          y_news = self.model_news(x_news)
        except:
          y_news = self.model_news.forward(x_news)

        x = torch.cat((y_num, y_news), dim=1)

        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)        
        x = self.fc4(x)

        return x

# complex model 2
# 6 layer linear multi layer after
class Complex_2(nn.Module):
    def __init__(self, model_num, model_news):  
        super(Complex_2, self).__init__()

        self.model_num = model_num
        self.model_news = model_news

        # create the NN after the two outputs
        # input -> trend probability from news and std. close prediction
        # output -> close prediction
        self.fc1 = nn.Linear(2,8)
        self.fc2 = nn.Linear(8,16)
        self.fc3 = nn.Linear(16,64)
        self.fc4 = nn.Linear(64,32)
        self.fc5 = nn.Linear(32,16)
        self.fc6 = nn.Linear(16,1)

    def forward(self, x_num, x_news):
        y_num = self.model_num(x_num)
        try:
          y_news = self.model_news(x_news)
        except:
          y_news = self.model_news.forward(x_news)

        x = torch.cat((y_num, y_news), dim=1)

        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)        
        x = self.fc4(x)
        x = self.fc5(x)        
        x = self.fc6(x)        

        return x      

# complex model 3
# 8 layer linear multi layer after
class Complex_3(nn.Module):
    def __init__(self, model_num, model_news):  
        super(Complex_3, self).__init__()

        self.model_num = model_num
        self.model_news = model_news


        # create the NN after the two outputs
        # input -> trend probability from news and std. close prediction
        # output -> close prediction
        self.fc1 = nn.Linear(2,8)
        self.fc2 = nn.Linear(8,32)
        self.fc3 = nn.Linear(32,64)
        self.fc4 = nn.Linear(64,256)
        self.fc5 = nn.Linear(256,128)
        self.fc6 = nn.Linear(128,64)
        self.fc7 = nn.Linear(64,16)
        self.fc8 = nn.Linear(16,1)  

        self.act = nn.ReLU()      

    def forward(self, x_num, x_news):
        y_num = self.model_num(x_num)
        try:
          y_news = self.model_news(x_news)
        except:
          y_news = self.model_news.forward(x_news)

        x = torch.cat((y_num, y_news), dim=1)

        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)        
        x = self.act(self.fc4(x))
        x = self.act(self.fc5(x))
        x = self.fc6(x)
        x = self.fc7(x)        
        x = self.fc8(x)        

        return x                      

# Train the models

In [ ]:
#save the time
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
def calc_score(prediction,real):
  prediction = torch.FloatTensor(prediction)
  real = torch.FloatTensor(real)
  score = 0
  loss = criterion(prediction,real)
  score = loss.item()
  score = score / len(prediction)
  return score   

In [ ]:
def save_the_log(path,name):
  # save the log
  with open(str(path)+"/"+str(name)+"_log.csv", "w") as f:
    s = ""      
    s += 'Model;Epoch;Accuraccy;Batch size;LR start;LR end;Dropout;Layer deepness;Activition function;Score\n'
    for i in range(len(model_name)):
      s += f'{model_name[i]};{e_res[i]};{acc_res[i]}%;{batch_s_res[i]};{optimizer_start_res[i]};{optimizer_end_res[i]};{dropout_res[i]};{layer_dep_res[i]};{act_res[i]};{score_res[i]}\n'
    
    f.write(s)

  plt_teach_res[-1].savefig(str(path)+"/"+str(name)+"_log_teach_" + str(model_name[-1]) + "_" + str(e_res[-1]) + ".png")
  try:
    plt_pred_res[-1].savefig(str(path)+"/"+str(name)+"_log_pred_" + str(model_name[-1]) + "_" + str(e_res[-1]) + ".png")     
  except:
    pass

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

## Numerical

In [ ]:
def run_model_num(batch_size_in, model_in, optimizer_in, criterion_in, model_name_in):
  # check if needed padding:
  if len(train_X_num) % batch_size_in == 0:
    pass
  else:
    res = int(len(train_X_num) / batch_size_in)
    needed_length = (res + 1) * batch_size_in
    pad = nn.ConstantPad2d((0,0,0,needed_length-len(train_X_num)),0)
    padded_train_X_num = pad(train_X_num)
    padded_train_Y_num = pad(train_Y_num)
    train_num_tensor = data_utils.TensorDataset(padded_train_X_num, padded_train_Y_num)
    train_num_loader = data_utils.DataLoader(dataset = train_num_tensor, 
                              batch_size = 32, shuffle = False)

  if len(valid_X_num) % batch_size_in == 0:
    pass
  else:
    res = int(len(valid_X_num) / batch_size_in)
    needed_length = (res + 1) * batch_size_in
    pad = nn.ConstantPad2d((0,0,0,needed_length-len(valid_X_num)),0)
    padded_valid_X_num = pad(valid_X_num)
    padded_valid_Y_num = pad(valid_Y_num)
    valid_num_tensor = data_utils.TensorDataset(padded_valid_X_num, padded_valid_Y_num)
    valid_num_loader = data_utils.DataLoader(dataset = valid_num_tensor, 
                              batch_size = 32, shuffle = False)                                 
  
  model = model_in

  optimizer = optimizer_in

  # Set the device
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  criterion = criterion_in

  model = model.to(device)
  criterion = criterion.to(device)

  # Training with Validation
  min_valid_loss = np.inf
  epoch_num = 1000

  #store the losses
  train_loss_array = []
  valid_loss_array = []
  epoch = 0

  real_test_array = []
  real_trend_array = test["Trend target"]  
  for y in test_Y_num:
    real_test_array.append(y*output_df.std()+output_df.mean())

  early_stop = False
  for e in range(epoch_num):
      start_time = time.time()

      train_loss = 0.0
      model.train()
      for x, y in train_num_loader:
          # Transfer Data to GPU if available
          if torch.cuda.is_available():
              x, y = x.cuda(), y.cuda()
            
          # Clear the gradients
          optimizer.zero_grad()
          # Forward Pass
          pred_y = model(x)
          # Find the Loss
          loss = criterion(pred_y,y)
          # Calculate gradients 
          loss.backward()
          # Update Weights
          optimizer.step()
          # Calculate Loss
          train_loss += loss.item() * x.size(0)

      train_loss = train_loss / len(train_num_loader.sampler)
      train_loss_array.append(train_loss) 

      valid_loss = 0.0
      model.eval()     # Optional when not using Model Specific layer
      for x, y in valid_num_loader:
          # Transfer Data to GPU if available
          if torch.cuda.is_available():
              x, y = x.cuda(), y.cuda()
            
          # Forward Pass
          pred_y = model(x)
          # Find the Loss
          loss = criterion(pred_y,y)
          # Calculate Loss
          valid_loss += loss.item() * x.size(0)
    
      valid_loss = valid_loss / len(valid_num_loader.sampler)
      valid_loss_array.append(valid_loss)

      end_time = time.time()

      epoch_mins, epoch_secs = epoch_time(start_time, end_time)
      
      print(f'Epoch {e+1} \t\t Epoch time: {epoch_mins}m {epoch_secs}s\n\t\t Training Loss: {train_loss} \t\t Validation Loss: {valid_loss}')
        
      if min_valid_loss > valid_loss:
          print(f'\t\t Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f}) \t Saving The Model')
                            
          # Saving State Dict
          torch.save(model.state_dict(), 'drive/MyDrive/complex/numerical_models/best_model_' + str(model_name_in) + '.pt')
          epoch = e

          # early stop
          if (valid_loss < 0.05 and train_loss < 0.05) and (min_valid_loss - valid_loss) < 0.00005:
              #early_stop = True
              pass

          min_valid_loss = valid_loss
        
      if  early_stop == True and e > 50:
          print(f'\tEarly stop of the training')
          break 

  # Visualize the training
  f1 = plt.figure(figsize=(16,8))
  plt.title('Train and validation loss')
  plt.plot(train_loss_array, color = "green", label = "Train loss")
  plt.plot(valid_loss_array, color = "blue", label = "Valid loss")
  plt.xlabel('Epoch',fontsize=18)
  plt.ylabel('Loss',fontsize=18)
  plt.legend(fontsize=18)
  plt_teach = f1
  plt.close()

  model.load_state_dict(torch.load('drive/MyDrive/complex/numerical_models/best_model_' + str(model_name_in) + '.pt'))

  predict_test_array = []

  with torch.no_grad():
      for x in test_X_num:
          model.eval()
          predict = model(x.reshape(1,-1).to(device))        
          predict_test_array.append(predict.item()*output_df.std()+output_df.mean()) 

  # calculate score
  score = calc_score(predict_test_array,real_test_array)    
  print(f"\nScore: {score}\n")      

  #check the trend -> create an array with the real and with the predicted trend
  #if the current value is bigger than before -> 1
  #otherwise (same or smaller) -> 0
  predicted_trend_array = []
  for element in range(len(predict_test_array)):
      real_today_close = test["Close"][element]*input_df["Close"].std()+input_df["Close"].mean()
      if real_today_close > predict_test_array[element].values:
          predicted_trend_array.append(0)
      else:
          predicted_trend_array.append(1)

  #check the number of differences
  trend_diff_array = []
  for element in range(len(real_trend_array)):
      if real_trend_array[element] != predicted_trend_array[element]:
        trend_diff_array.append(element)

  #percentage of good predict
  acc = (len(real_trend_array)-len(trend_diff_array))/len(real_trend_array)*100
  print(f"Accuracy: {acc}\n")  
  #visualize it
  f2 = plt.figure(figsize=(24,12))
  plt.title("Predicted and real close prices", fontsize = 18)
  plt.plot(real_test_array, color = "blue", linewidth = 4,
          label = "Real")
  plt.plot(predict_test_array, color = "red", linewidth = 2,
          label = "Predicted")
  plt.xlabel("Date",fontsize = 18)
  plt.legend(fontsize = 18)
  f2.set_size_inches(12,6)
  plt_best = f2
  plt.close()

  # return the results for compare:
  #   epoch
  #   trend result (best)
  #   diagram (best)
  #   score
  return epoch, score, acc, plt_teach, plt_best

In [ ]:
model = []
model.append(LSTM_92_num())

for i in range(len(model)):
    model_name = ["LSTM_92_num"]
    optimizer = optim.Adam(model[i].parameters(), 0.0001)
    criterion = nn.MSELoss()

    e_res = []
    acc_res = []
    batch_s_res = []
    optimizer_start_res = []
    optimizer_end_res = []
    dropout_res = []
    layer_dep_res = []
    act_res = []
    score_res = []
    plt_teach_res = []
    plt_pred_res = []

    epoch_out, score_out, acc_out, plt_teach_out, plt_best_out = run_model_num(batch_size_in=32,model_in=model[i],
          optimizer_in=optimizer,criterion_in=criterion,model_name_in=model_name[0])

    e_res.append(epoch_out)
    acc_res.append(acc_out)
    batch_s_res.append("32")
    optimizer_start_res.append("0.0001")
    optimizer_end_res.append("0.0001")
    dropout_res.append("0.1")
    layer_dep_res.append("4")
    act_res.append("None")
    score_res.append(score_out)
    plt_teach_res.append(plt_teach_out)
    plt_pred_res.append(plt_best_out)

    save_the_log(path="drive/MyDrive/complex/numerical_models",name=model_name[0])

Epoch 1 		 Epoch time: 0m 1s
		 Training Loss: 0.4904472491628415 		 Validation Loss: 1.4334435371252208
		 Validation Loss Decreased(inf--->1.433444) 	 Saving The Model
Epoch 2 		 Epoch time: 0m 1s
		 Training Loss: 0.417036327621522 		 Validation Loss: 1.3912824300619273
		 Validation Loss Decreased(1.433444--->1.391282) 	 Saving The Model
Epoch 3 		 Epoch time: 0m 1s
		 Training Loss: 0.29295839901661147 		 Validation Loss: 1.1803890054042523
		 Validation Loss Decreased(1.391282--->1.180389) 	 Saving The Model
Epoch 4 		 Epoch time: 0m 1s
		 Training Loss: 0.12919648620299995 		 Validation Loss: 0.897739419570336
		 Validation Loss Decreased(1.180389--->0.897739) 	 Saving The Model
Epoch 5 		 Epoch time: 0m 1s
		 Training Loss: 0.060284077158399126 		 Validation Loss: 0.7773278745321127
		 Validation Loss Decreased(0.897739--->0.777328) 	 Saving The Model
Epoch 6 		 Epoch time: 0m 1s
		 Training Loss: 0.04634876920514413 		 Validation Loss: 0.6216224546615894
		 Validation Loss Dec

In [ ]:
model_name = ["Linear_108"]
model = Linear_108_num()

optimizer = optim.Adam(model.parameters(), 0.0001)
criterion = nn.MSELoss()

e_res = []
acc_res = []
batch_s_res = []
optimizer_start_res = []
optimizer_end_res = []
dropout_res = []
layer_dep_res = []
act_res = []
score_res = []
plt_teach_res = []
plt_pred_res = []

In [ ]:
epoch_out, score_out, acc_out, plt_teach_out, plt_best_out = run_model_num(batch_size_in=32,model_in=model,
          optimizer_in=optimizer,criterion_in=criterion,model_name_in=model_name[0])

e_res.append(epoch_out)
acc_res.append(acc_out)
batch_s_res.append("32")
optimizer_start_res.append("0.0001")
optimizer_end_res.append("0.0001")
dropout_res.append("0")
layer_dep_res.append("11")
act_res.append("None")
score_res.append(score_out)
plt_teach_res.append(plt_teach_out)
plt_pred_res.append(plt_best_out)

Epoch 1 		 Epoch time: 0m 0s
		 Training Loss: 0.8198406951300599 		 Validation Loss: 0.0977781331883027
		 Validation Loss Decreased(inf--->0.097778) 	 Saving The Model
Epoch 2 		 Epoch time: 0m 0s
		 Training Loss: 0.03750567631544294 		 Validation Loss: 0.013064355481989108
		 Validation Loss Decreased(0.097778--->0.013064) 	 Saving The Model
Epoch 3 		 Epoch time: 0m 0s
		 Training Loss: 0.0092601490936972 		 Validation Loss: 0.015160845807538582
Epoch 4 		 Epoch time: 0m 0s
		 Training Loss: 0.0043946910835173285 		 Validation Loss: 0.0034738015652132723
		 Validation Loss Decreased(0.013064--->0.003474) 	 Saving The Model
Epoch 5 		 Epoch time: 0m 0s
		 Training Loss: 0.0030229160911403596 		 Validation Loss: 0.0017206401680596173
		 Validation Loss Decreased(0.003474--->0.001721) 	 Saving The Model
Epoch 6 		 Epoch time: 0m 0s
		 Training Loss: 0.0028252919610413547 		 Validation Loss: 0.002196418985616989
Epoch 7 		 Epoch time: 0m 0s
		 Training Loss: 0.002913143123166535 		 Va

In [ ]:
save_the_log(path="drive/MyDrive/complex/numerical_models",name=model_name[0])

## News

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(preds)
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def runLogReg(batch_size_in, model_in, model_name_in):
  # check if needed padding:
  if len(train_X_news) % batch_size_in == 0:
    pass
  else:
    res = int(len(train_X_news) / batch_size_in)
    needed_length = (res + 1) * batch_size_in
    pad = nn.ConstantPad2d((0,0,0,needed_length-len(train_X_news)),0)
    padded_train_X_news = pad(train_X_news)
    padded_train_Y_news = pad(train_Y_news)
    train_news_tensor = data_utils.TensorDataset(padded_train_X_news, padded_train_Y_news)
    train_news_loader = data_utils.DataLoader(dataset = train_news_tensor, 
                              batch_size = 32, shuffle = False)

  if len(valid_X_news) % batch_size_in == 0:
    pass
  else:
    res = int(len(valid_X_news) / batch_size_in)
    needed_length = (res + 1) * batch_size_in
    pad = nn.ConstantPad2d((0,0,0,needed_length-len(valid_X_news)),0)
    padded_valid_X_news = pad(valid_X_news)
    padded_valid_Y_news = pad(valid_Y_news)
    valid_news_tensor = data_utils.TensorDataset(padded_valid_X_news, padded_valid_Y_news)
    valid_news_loader = data_utils.DataLoader(dataset = valid_news_tensor, 
                              batch_size = 32, shuffle = False)                                 
  
  model = model_in

  # train the model
  model.train(train_X_news, train_Y_news)

  # calculate accuraccy
  global predict_test_array
  predict_test_array = []

  for x in test_X_news:
      predict = model.forward(x)
      predict_test_array.append(predict[0]) 

  test_acc = binary_accuracy(torch.FloatTensor(predict_test_array).reshape(len(predict_test_array),1),test_Y_news)  
  acc = test_acc*100
  print(f"\nAccuracy: {acc}%\n")

  # round the prediction
  rounded = []
  for prediction in predict_test_array:
      rounded.append(round(prediction))

  print(pd.crosstab(torch.reshape(test_Y_news,(-1,)), np.array(rounded), rownames=["Actual"], colnames=["Predicted"]))
  print()
  print(classification_report(torch.reshape(test_Y_news,(-1,)), rounded))
  print(accuracy_score(torch.reshape(test_Y_news,(-1,)), rounded))

  # save the model to disk
  filename = 'drive/MyDrive/complex/news_models/best_model_' + str(model_name_in) + '.pt'
  pickle.dump(model, open(filename, 'wb'))

In [ ]:
model_logreg = LogReg()
runLogReg(batch_size_in=32, model_in=model_logreg, model_name_in="LogReg")

Train done

Accuracy: 51.776649475097656%

Predicted   0    1
Actual            
0.0        86  107
1.0        83  118

              precision    recall  f1-score   support

         0.0       0.51      0.45      0.48       193
         1.0       0.52      0.59      0.55       201

    accuracy                           0.52       394
   macro avg       0.52      0.52      0.51       394
weighted avg       0.52      0.52      0.52       394

0.5177664974619289


In [ ]:
def runVADER(batch_size_in, model_in):
  # check if needed padding:
  if len(train_X_news) % batch_size_in == 0:
    pass
  else:
    res = int(len(train_X_news) / batch_size_in)
    needed_length = (res + 1) * batch_size_in
    pad = nn.ConstantPad2d((0,0,0,needed_length-len(train_X_news)),0)
    padded_train_X_news = pad(train_X_news)
    padded_train_Y_news = pad(train_Y_news)
    train_news_tensor = data_utils.TensorDataset(padded_train_X_news, padded_train_Y_news)
    train_news_loader = data_utils.DataLoader(dataset = train_news_tensor, 
                              batch_size = 32, shuffle = False)

  if len(valid_X_news) % batch_size_in == 0:
    pass
  else:
    res = int(len(valid_X_news) / batch_size_in)
    needed_length = (res + 1) * batch_size_in
    pad = nn.ConstantPad2d((0,0,0,needed_length-len(valid_X_news)),0)
    padded_valid_X_news = pad(valid_X_news)
    padded_valid_Y_news = pad(valid_Y_news)
    valid_news_tensor = data_utils.TensorDataset(padded_valid_X_news, padded_valid_Y_news)
    valid_news_loader = data_utils.DataLoader(dataset = valid_news_tensor, 
                              batch_size = 32, shuffle = False)                                 
  
  model = model_in

  # calculate accuraccy
  predict_test_array = []

  for x in test_X_news:
      predict = model.forward(x)
      predict_test_array.append(predict) 

  test_acc = binary_accuracy(torch.FloatTensor(predict_test_array).reshape(len(predict_test_array),1),test_Y_news)  
  acc = test_acc*100
  print(f"\tAccuracy: {acc}%\n")

In [ ]:
model_vader = VADER()
runVADER(batch_size_in=32, model_in=model_vader)

	Accuracy: 50.76142120361328%



In [ ]:
def run_news_model(batch_size_in, model_in, optimizer_in, criterion_in, model_name_in):
  # check if needed padding:
  if len(train_X_news) % batch_size_in == 0:
    pass
  else:
    res = int(len(train_X_news) / batch_size_in)
    needed_length = (res + 1) * batch_size_in
    pad = nn.ConstantPad2d((0,0,0,needed_length-len(train_X_news)),0)
    padded_train_X_news = pad(train_X_news)
    padded_train_Y_news = pad(train_Y_news)
    train_news_tensor = data_utils.TensorDataset(padded_train_X_news, padded_train_Y_news)
    train_news_loader = data_utils.DataLoader(dataset = train_news_tensor, 
                              batch_size = 32, shuffle = False)

  if len(valid_X_news) % batch_size_in == 0:
    pass
  else:
    res = int(len(valid_X_news) / batch_size_in)
    needed_length = (res + 1) * batch_size_in
    pad = nn.ConstantPad2d((0,0,0,needed_length-len(valid_X_news)),0)
    padded_valid_X_news = pad(valid_X_news)
    padded_valid_Y_news = pad(valid_Y_news)
    valid_news_tensor = data_utils.TensorDataset(padded_valid_X_news, padded_valid_Y_news)
    valid_news_loader = data_utils.DataLoader(dataset = valid_news_tensor, 
                              batch_size = 32, shuffle = False)                                 
  
  model = model_in

  optimizer = optimizer_in

  # Set the device
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  criterion = criterion_in

  model = model.to(device)
  criterion = criterion.to(device)

  # Training with Validation
  min_valid_loss = np.inf
  epoch_num = 75

  #store the losses
  train_loss_array = []
  train_acc_array = []
  valid_loss_array = []
  valid_acc_array = []
  epoch = 0

  early_stop = False
  for e in range(epoch_num):
      start_time = time.time()

      train_loss = 0.0
      train_acc = 0.0
      model.train()
      for x, y in train_news_loader: # x is news, y is trend target
          # Transfer Data to GPU if available
          if torch.cuda.is_available():
              x, y = x.cuda(), y.cuda()
            
          # Clear the gradients
          optimizer.zero_grad()
          # Forward Pass
          pred_y = model(x)
          # Find the Loss
          loss = criterion(pred_y,y)
          # Calculate gradients 
          loss.backward()
          # Update Weights
          optimizer.step()
          # Calculate Loss
          train_loss += loss.item() * x.size(0)
          # Calculate acc
          train_acc += binary_accuracy(pred_y, y).item() * x.size(0)


      train_loss = train_loss / len(train_news_loader.sampler)
      train_loss_array.append(train_loss) 

      train_acc = train_acc / len(train_news_loader.sampler)
      train_acc_array.append(train_acc)

      valid_loss = 0.0
      valid_acc = 0.0
      model.eval()     # Optional when not using Model Specific layer
      for x, y in valid_news_loader:
          # Transfer Data to GPU if available
          if torch.cuda.is_available():
              x, y = x.cuda(), y.cuda()
            
          # Forward Pass
          pred_y = model(x)
          # Find the Loss
          loss = criterion(pred_y,y)
          # Calculate Loss
          valid_loss += loss.item() * x.size(0)
          # Calculate acc
          valid_acc += binary_accuracy(pred_y, y).item() * x.size(0)
    
      valid_loss = valid_loss / len(valid_news_loader.sampler)
      valid_loss_array.append(valid_loss)

      valid_acc = valid_acc / len(valid_news_loader.sampler)
      valid_acc_array.append(valid_acc)      

      end_time = time.time()

      epoch_mins, epoch_secs = epoch_time(start_time, end_time)
      
      print(f'Epoch {e+1} \t\t Epoch time: {epoch_mins}m {epoch_secs}s\n\t\t Training Loss: {train_loss} \t\t Validation Loss: {valid_loss}')
      print(f'\tTrain Acc: {train_acc*100:.2f}% \t\ Val. Acc: {valid_acc*100:.2f}%')
        
      if min_valid_loss > valid_loss:
          print(f'\t\t Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f}) \t Saving The Model')
                            
          # Saving State Dict
          torch.save(model.state_dict(), 'drive/MyDrive/complex/news_models/best_model_' + str(model_name_in) + '.pt')
          epoch = e

          # early stop
          if valid_loss < 0.6 and (min_valid_loss - valid_loss) < 0.05:
              early_stop = True

          min_valid_loss = valid_loss

      if (valid_loss > 0.7 and train_loss < 0.6):
          early_stop = True
        
      if  early_stop == True and e > 10:
          print(f'\tEarly stop of the training')
          break 

      # Saving State Dict
      torch.save(model.state_dict(), 'drive/MyDrive/complex/news_models/' + str(e) + '_model_' + str(model_name_in) + '.pt')  

      # Saving State Dict
      torch.save(model.state_dict(), 'drive/MyDrive/complex/news_models/last_model_' + str(model_name_in) + '.pt')     

  # Visualize the training
  f1 = plt.figure(figsize=(16,8))
  plt.title('Train and validation loss')
  plt.plot(train_loss_array, color = "green", label = "Train loss")
  plt.plot(valid_loss_array, color = "blue", label = "Valid loss")
  plt.xlabel('Epoch',fontsize=18)
  plt.ylabel('Loss',fontsize=18)
  plt.legend(fontsize=18)
  plt_teach = f1
  plt.close()

  model.load_state_dict(torch.load('drive/MyDrive/complex/news_models/best_model_' + str(model_name_in) + '.pt'))  

  # calculate accuraccy
  predict_test_array = []

  with torch.no_grad():
      for x in test_X_news:
          model.eval()
          predict = model(torch.reshape(x, (1, len(x))).cuda())     # reshape for BERT   
          predict_test_array.append(predict) 

  test_acc = binary_accuracy(torch.FloatTensor(predict_test_array).reshape(len(predict_test_array),1),test_Y_news)  
  acc = test_acc*100
  print(f"\tAccuracy: {acc}%\n")  

  # return the results for compare:
  #   epoch
  #   trend result (best)
  #   diagram (best)
  return epoch, acc, plt_teach

In [ ]:
# pretrain bert -> can be merge with the other cells later
model_name = ["BERT"]
model = BERT_news()
print(f'The model has {count_parameters(model):,} trainable parameters')

# freeze the embedding layer / bert layer
for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False

print(f'The model has {count_parameters(model):,} trainable parameters')

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCEWithLogitsLoss()

e_res = []
acc_res = []
batch_s_res = []
optimizer_start_res = []
optimizer_end_res = []
dropout_res = []
layer_dep_res = []
act_res = []
score_res = []
plt_teach_res = []
plt_pred_res = []

epoch_out, acc_out, plt_teach_out = run_news_model(batch_size_in=32,model_in=model,
          optimizer_in=optimizer,criterion_in=criterion,model_name_in=model_name[0])

e_res.append(epoch_out)
acc_res.append(acc_out)
batch_s_res.append("32")
optimizer_start_res.append("0.001")
optimizer_end_res.append("0.001")
dropout_res.append("0.3")
layer_dep_res.append("4")
act_res.append("Sigmoid")
score_res.append("NA")
plt_teach_res.append(plt_teach_out)
plt_pred_res.append("None")

save_the_log(path="drive/MyDrive/complex/news_models",name=model_name[0])

The model has 83,361 trainable parameters
The model has 83,361 trainable parameters
Epoch 1 		 Epoch time: 2m 1s
		 Training Loss: 0.6970433045077968 		 Validation Loss: 0.6970557937255273
	Train Acc: 45.19% 	\ Val. Acc: 47.60%
		 Validation Loss Decreased(inf--->0.697056) 	 Saving The Model
Epoch 2 		 Epoch time: 2m 0s
		 Training Loss: 0.6934060808774587 		 Validation Loss: 0.6952424003527715
	Train Acc: 46.11% 	\ Val. Acc: 47.60%
		 Validation Loss Decreased(0.697056--->0.695242) 	 Saving The Model
Epoch 3 		 Epoch time: 2m 0s
		 Training Loss: 0.6924466216886366 		 Validation Loss: 0.6943293167994573
	Train Acc: 46.11% 	\ Val. Acc: 47.60%
		 Validation Loss Decreased(0.695242--->0.694329) 	 Saving The Model
Epoch 4 		 Epoch time: 2m 0s
		 Training Loss: 0.6911123829918939 		 Validation Loss: 0.6938860416412354
	Train Acc: 46.11% 	\ Val. Acc: 47.60%
		 Validation Loss Decreased(0.694329--->0.693886) 	 Saving The Model
Epoch 5 		 Epoch time: 2m 0s
		 Training Loss: 0.6911462174879538

In [ ]:
for e in range(75):
    model.load_state_dict(torch.load(f'drive/MyDrive/complex/news_models/{e}_model_' + str(model_name[0]) + '.pt'))  

    # calculate accuraccy
    predict_test_array = []

    with torch.no_grad():
        for x in test_X_news:
            model.eval()
            predict = model(torch.reshape(x, (1, len(x))).cuda())        
            predict_test_array.append(predict) 

    test_acc = binary_accuracy(torch.FloatTensor(predict_test_array).reshape(len(predict_test_array),1),test_Y_news)  
    acc = test_acc*100
    print(f"{e}. epoch\n")
    print(f"Accuracy: {acc}%\n")
    print("------------------\n")

0. epoch

Accuracy: 48.984771728515625%

------------------

1. epoch

Accuracy: 48.984771728515625%

------------------

2. epoch

Accuracy: 48.984771728515625%

------------------

3. epoch

Accuracy: 48.984771728515625%

------------------

4. epoch

Accuracy: 48.984771728515625%

------------------

5. epoch

Accuracy: 48.984771728515625%

------------------

6. epoch

Accuracy: 48.984771728515625%

------------------

7. epoch

Accuracy: 49.23857879638672%

------------------

8. epoch

Accuracy: 48.984771728515625%

------------------

9. epoch

Accuracy: 48.984771728515625%

------------------

10. epoch

Accuracy: 47.96954345703125%

------------------

11. epoch

Accuracy: 49.23857879638672%

------------------

12. epoch

Accuracy: 48.984771728515625%

------------------

13. epoch

Accuracy: 48.73096466064453%

------------------

14. epoch

Accuracy: 48.73096466064453%

------------------

15. epoch

Accuracy: 48.47715759277344%

------------------

16. epoch

Accuracy: 48.

In [ ]:
model_name = ["BERT"]
model = BERT_news()
print(f'The model has {count_parameters(model):,} trainable parameters')

# freeze the embedding layer / bert layer
for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 109,565,601 trainable parameters
The model has 83,361 trainable parameters


In [ ]:
for e in range(52,75):
    model.load_state_dict(torch.load(f'drive/MyDrive/complex/news_models/{e}_model_' + str(model_name[0]) + '.pt', map_location=torch.device('cpu')))  

    # calculate accuraccy
    predict_test_array = []

    with torch.no_grad():
        for x in test_X_news:
            model.eval()
            predict = model(torch.reshape(x, (1, len(x))))        
            predict_test_array.append(predict) 

    test_acc = binary_accuracy(torch.FloatTensor(predict_test_array).reshape(len(predict_test_array),1),test_Y_news)  
    acc = test_acc*100
    print(f"{e}. epoch\n")
    print(f"Accuracy: {acc}%\n")
    print("------------------\n")

52. epoch

Accuracy: 48.984771728515625%

------------------

53. epoch

Accuracy: 48.73096466064453%

------------------

54. epoch

Accuracy: 48.984771728515625%

------------------

55. epoch

Accuracy: 48.984771728515625%

------------------

56. epoch

Accuracy: 49.23857879638672%

------------------

57. epoch

Accuracy: 48.984771728515625%

------------------

58. epoch

Accuracy: 49.23857879638672%

------------------

59. epoch

Accuracy: 49.23857879638672%

------------------

60. epoch

Accuracy: 48.984771728515625%

------------------

61. epoch

Accuracy: 51.26903533935547%

------------------

62. epoch

Accuracy: 52.284263610839844%

------------------

63. epoch

Accuracy: 52.5380744934082%

------------------

64. epoch

Accuracy: 51.26903533935547%

------------------

65. epoch

Accuracy: 48.73096466064453%

------------------

66. epoch

Accuracy: 49.746192932128906%

------------------

67. epoch

Accuracy: 48.984771728515625%

------------------

68. epoch

Accura

In [ ]:
e = 75
model.load_state_dict(torch.load(f'drive/MyDrive/complex/news_models/{e}_model_' + str(model_name[0]) + '.pt', map_location=torch.device('cpu')))  

# calculate accuraccy
predict_test_array = []

with torch.no_grad():
    for x in test_X_news:
        model.eval()
        predict = model(torch.reshape(x, (1, len(x))))        
        predict_test_array.append(predict) 

test_acc = binary_accuracy(torch.FloatTensor(predict_test_array).reshape(len(predict_test_array),1),test_Y_news)  
acc = test_acc*100
print(f"{e}. epoch\n")
print(f"Accuracy: {acc}%\n")
print("------------------\n")

In [ ]:
PAD_ID = vocab.lookup_indices(["<pad>"])[0]

model_name = ["LSTM_pilot"]
model = LSTM_news(len(vocab),PAD_ID)
print(f'The model has {count_parameters(model):,} trainable parameters')

# copy the pretrained embedding weight
model.embedding.weight.data.copy_(pretrained_embedding)

# freeze the embedding layer
model.embedding.weight.requires_grad = False
print(f'The model has {count_parameters(model):,} trainable parameters')

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCEWithLogitsLoss()

e_res = []
acc_res = []
batch_s_res = []
optimizer_start_res = []
optimizer_end_res = []
dropout_res = []
layer_dep_res = []
act_res = []
score_res = []
plt_teach_res = []
plt_pred_res = []

The model has 156,745 trainable parameters
The model has 48,945 trainable parameters


In [ ]:
epoch_out, acc_out, plt_teach_out = run_news_model(batch_size_in=32,model_in=model,
          optimizer_in=optimizer,criterion_in=criterion,model_name_in=model_name[0])

e_res.e_resappend(epoch_out)
acc_res.append(acc_out)
batch_s_res.append("32")
optimizer_start_res.append("0.001")
optimizer_end_res.append("0.001")
dropout_res.append("0.33")
layer_dep_res.append("6")
act_res.append("Sigmoid")
score_res.append("NA")
plt_teach_res.append(plt_teach_out)
plt_pred_res.append("None")

save_the_log(path="drive/MyDrive/complex/news_models",name=model_name[0])

Epoch 1 		 Epoch time: 0m 14s
		 Training Loss: 0.7024878698426325 		 Validation Loss: 0.7062262159127456
	Train Acc: 46.11% 	\ Val. Acc: 47.60%
		 Validation Loss Decreased(inf--->0.706226) 	 Saving The Model
Epoch 2 		 Epoch time: 0m 14s
		 Training Loss: 0.6958418021330962 		 Validation Loss: 0.6946915021309485
	Train Acc: 46.11% 	\ Val. Acc: 47.60%
		 Validation Loss Decreased(0.706226--->0.694692) 	 Saving The Model
Epoch 3 		 Epoch time: 0m 14s
		 Training Loss: 0.6904327740540376 		 Validation Loss: 0.6926051011452308
	Train Acc: 46.11% 	\ Val. Acc: 47.60%
		 Validation Loss Decreased(0.694692--->0.692605) 	 Saving The Model
Epoch 4 		 Epoch time: 0m 14s
		 Training Loss: 0.6904663089159373 		 Validation Loss: 0.6923538263027484
	Train Acc: 46.11% 	\ Val. Acc: 47.60%
		 Validation Loss Decreased(0.692605--->0.692354) 	 Saving The Model
Epoch 5 		 Epoch time: 0m 14s
		 Training Loss: 0.6901416230846096 		 Validation Loss: 0.6923527671740606
	Train Acc: 46.11% 	\ Val. Acc: 47.60%


In [ ]:
for e in range(30):
    model.load_state_dict(torch.load(f'drive/MyDrive/complex/news_models/{e}_model_' + str(model_name[0]) + '.pt'))  

    # calculate accuraccy
    predict_test_array = []

    with torch.no_grad():
        for x in test_X_news:
            model.eval()
            predict = model(x.to("cuda"))        
            predict_test_array.append(predict) 

    test_acc = binary_accuracy(torch.FloatTensor(predict_test_array).reshape(len(predict_test_array),1),test_Y_news)  
    acc = test_acc*100
    print(f"{e}. epoch\n")
    print(f"Accuracy: {acc}%\n")
    print("------------------\n")

0. epoch

Accuracy: 48.984771728515625%

------------------

1. epoch

Accuracy: 48.984771728515625%

------------------

2. epoch

Accuracy: 48.984771728515625%

------------------

3. epoch

Accuracy: 48.984771728515625%

------------------

4. epoch

Accuracy: 48.984771728515625%

------------------

5. epoch

Accuracy: 48.984771728515625%

------------------

6. epoch

Accuracy: 48.984771728515625%

------------------

7. epoch

Accuracy: 48.984771728515625%

------------------

8. epoch

Accuracy: 48.984771728515625%

------------------

9. epoch

Accuracy: 48.984771728515625%

------------------

10. epoch

Accuracy: 48.984771728515625%

------------------

11. epoch

Accuracy: 48.984771728515625%

------------------

12. epoch

Accuracy: 50.0%

------------------

13. epoch

Accuracy: 52.5380744934082%

------------------

14. epoch

Accuracy: 52.284263610839844%

------------------

15. epoch

Accuracy: 51.52284622192383%

------------------

16. epoch

Accuracy: 52.79187393188

## Complex

In [ ]:
def run_model_complex(batch_size_in, model_in, optimizer_in, criterion_in, model_name_in):
  # check if needed padding:
  if len(train_X_num) % batch_size_in == 0:
    pass
  else:
    res = int(len(train_X_num) / batch_size_in)
    needed_length = (res + 1) * batch_size_in
    pad = nn.ConstantPad2d((0,0,0,needed_length-len(train_X_num)),0)
    padded_train_X_num = pad(train_X_num)
    padded_train_X_news = pad(train_X_news)
    padded_train_Y_num = pad(train_Y_num)
    train_all_tensor = data_utils.TensorDataset(padded_train_X_num, padded_train_X_news, padded_train_Y_num)
    train_all_loader = data_utils.DataLoader(dataset = train_all_tensor, 
                              batch_size = 32, shuffle = False)

  if len(valid_X_num) % batch_size_in == 0:
    pass
  else:
    res = int(len(valid_X_num) / batch_size_in)
    needed_length = (res + 1) * batch_size_in
    pad = nn.ConstantPad2d((0,0,0,needed_length-len(valid_X_num)),0)
    padded_valid_X_num = pad(valid_X_num)
    padded_valid_X_news = pad(valid_X_news)    
    padded_valid_Y_num = pad(valid_Y_num)
    valid_all_tensor = data_utils.TensorDataset(padded_valid_X_num, padded_valid_X_news, padded_valid_Y_num)
    valid_all_loader = data_utils.DataLoader(dataset = valid_all_tensor, 
                              batch_size = 32, shuffle = False)                                 
  
  model = model_in

  optimizer = optimizer_in

  # Set the device
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  criterion = criterion_in

  model = model.to(device)
  criterion = criterion.to(device)

  # Training with Validation
  min_valid_loss = np.inf
  #epoch_num = 200
  epoch_num = 75 # bert

  #store the losses
  train_loss_array = []
  valid_loss_array = []
  epoch = 0

  real_test_array = []
  real_trend_array = test["Trend target"]  
  for y in test_Y_num:
    real_test_array.append(y*output_df.std()+output_df.mean())

  early_stop = False
  for e in range(epoch_num):
      start_time = time.time()

      train_loss = 0.0
      model.train()
      for x_num, x_text, y in train_all_loader:
          # Transfer Data to GPU if available
          if torch.cuda.is_available():
              x_num, x_text, y = x_num.cuda(), x_text.cuda(), y.cuda()
            
          # Clear the gradients
          optimizer.zero_grad()
          # Forward Pass
          pred_y = model(x_num, x_text)
          # Find the Loss
          loss = criterion(pred_y,y)
          # Calculate gradients 
          loss.backward()
          # Update Weights
          optimizer.step()
          # Calculate Loss
          train_loss += loss.item() * x_num.size(0)

      train_loss = train_loss / len(train_all_loader.sampler)
      train_loss_array.append(train_loss) 

      valid_loss = 0.0
      model.eval()     # Optional when not using Model Specific layer
      for x_num, x_text, y in valid_all_loader:
          # Transfer Data to GPU if available
          if torch.cuda.is_available():
              x_num, x_text, y = x_num.cuda(), x_text.cuda(), y.cuda()
            
          # Forward Pass
          pred_y = model(x_num, x_text)
          # Find the Loss
          loss = criterion(pred_y,y)
          # Calculate Loss
          valid_loss += loss.item() * x_num.size(0)
    
      valid_loss = valid_loss / len(valid_all_loader.sampler)
      valid_loss_array.append(valid_loss)

      end_time = time.time()

      epoch_mins, epoch_secs = epoch_time(start_time, end_time)
      
      print(f'Epoch {e+1} \t\t Epoch time: {epoch_mins}m {epoch_secs}s\n\t\t Training Loss: {train_loss} \t\t Validation Loss: {valid_loss}')
        
      if min_valid_loss > valid_loss:
          print(f'\t\t Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f}) \t Saving The Model')
                            
          # Saving State Dict
          torch.save(model.state_dict(), 'drive/MyDrive/complex/complex_models/best_model_' + str(model_name_in) + '.pt')
          epoch = e

          # early stop
          if (valid_loss < 0.05 and train_loss < 0.05) and (min_valid_loss - valid_loss) < 0.0005:
              early_stop = True

          min_valid_loss = valid_loss
        
      if  early_stop == True and e > 25: # for bert
          print(f'\tEarly stop of the training')
          break 

      # Saving State Dict
      #torch.save(model.state_dict(), f'drive/MyDrive/complex/complex_models/{e}_model_' + str(model_name_in) + '.pt') -> only best for BERT
      #epoch = e    

  # Visualize the training
  f1 = plt.figure(figsize=(16,8))
  plt.title('Train and validation loss')
  plt.plot(train_loss_array, color = "green", label = "Train loss")
  plt.plot(valid_loss_array, color = "blue", label = "Valid loss")
  plt.xlabel('Epoch',fontsize=18)
  plt.ylabel('Loss',fontsize=18)
  plt.legend(fontsize=18)
  plt_teach = f1
  plt.close()

  model.load_state_dict(torch.load('drive/MyDrive/complex/complex_models/best_model_' + str(model_name_in) + '.pt'))

  predict_test_array = []

  with torch.no_grad():
      for i in range(len(test_X_num)):
          model.eval()
          predict = model(test_X_num[i].reshape(1,-1).to(device), test_X_news[i].reshape(1,-1).to(device))        
          predict_test_array.append(predict.item()*output_df.std()+output_df.mean()) 

  # calculate score
  score = calc_score(predict_test_array,real_test_array)    
  print(f"\nScore: {score}\n")      

  #check the trend -> create an array with the real and with the predicted trend
  #if the current value is bigger than before -> 1
  #otherwise (same or smaller) -> 0
  predicted_trend_array = []
  for element in range(len(predict_test_array)):
      real_today_close = test["Close"][element]*input_df["Close"].std()+input_df["Close"].mean()
      if real_today_close > predict_test_array[element].values:
          predicted_trend_array.append(0)
      else:
          predicted_trend_array.append(1)

  #check the number of differences
  trend_diff_array = []
  for element in range(len(real_trend_array)):
      if real_trend_array[element] != predicted_trend_array[element]:
        trend_diff_array.append(element)

  #percentage of good predict
  acc = (len(real_trend_array)-len(trend_diff_array))/len(real_trend_array)*100
  print(f"Accuracy: {acc}\n")  
  #visualize it
  f2 = plt.figure(figsize=(24,12))
  plt.title("Predicted and real close prices", fontsize = 18)
  plt.plot(real_test_array, color = "blue", linewidth = 4,
          label = "Real")
  plt.plot(predict_test_array, color = "red", linewidth = 2,
          label = "Predicted")
  plt.xlabel("Date",fontsize = 18)
  plt.legend(fontsize = 18)
  f2.set_size_inches(12,6)
  plt_best = f2
  plt.close()

  # return the results for compare:
  #   epoch
  #   trend result (best)
  #   diagram (best)
  #   score
  return epoch, score, acc, plt_teach, plt_best

In [ ]:
model_name = ["Complex_1"]

# create and load models
model_num = Linear_108_num()
model_num.load_state_dict(torch.load('drive/MyDrive/complex/numerical_models/best_model_Linear_108.pt'))

PAD_ID = vocab.lookup_indices(["<pad>"])[0]
model_news = LSTM_news(len(vocab),PAD_ID)
model_news.load_state_dict(torch.load(f'drive/MyDrive/complex/news_models/26_model_LSTM_pilot.pt'))

model = Complex_1(model_num, model_news)

# freeze the numerical and news weights
print(f'The model has {count_parameters(model):,} trainable parameters')

for p in model.model_num.parameters():
    p.requires_grad = False
for p in model.model_news.parameters():
    p.requires_grad = False
print(f'The model has {count_parameters(model):,} trainable parameters')

optimizer = optim.Adam(model.parameters(), 0.0001)
criterion = nn.MSELoss()

e_res = []
acc_res = []
batch_s_res = []
optimizer_start_res = []
optimizer_end_res = []
dropout_res = []
layer_dep_res = []
act_res = []
score_res = []
plt_teach_res = []
plt_pred_res = []

The model has 1,555,011 trainable parameters
The model has 857 trainable parameters


In [ ]:
epoch_out, score_out, acc_out, plt_teach_out, plt_best_out = run_model_complex(batch_size_in=32,model_in=model,
          optimizer_in=optimizer,criterion_in=criterion,model_name_in=model_name[0])

e_res.append(epoch_out)
acc_res.append(acc_out)
batch_s_res.append("32")
optimizer_start_res.append("0.0001")
optimizer_end_res.append("0.0001")
dropout_res.append("0")
layer_dep_res.append("6")
act_res.append("None")
score_res.append(score_out)
plt_teach_res.append(plt_teach_out)
plt_pred_res.append(plt_best_out)

Epoch 1 		 Epoch time: 0m 6s
		 Training Loss: 0.595920604710291 		 Validation Loss: 0.9398433359769675
		 Validation Loss Decreased(inf--->0.939843) 	 Saving The Model
Epoch 2 		 Epoch time: 0m 6s
		 Training Loss: 0.5258329630879736 		 Validation Loss: 1.0004539558520684
Epoch 3 		 Epoch time: 0m 6s
		 Training Loss: 0.4643050474995697 		 Validation Loss: 1.048401727126195
Epoch 4 		 Epoch time: 0m 6s
		 Training Loss: 0.4092565536801074 		 Validation Loss: 1.0818303502522981
Epoch 5 		 Epoch time: 0m 7s
		 Training Loss: 0.3617907448898296 		 Validation Loss: 1.0987170751278217
Epoch 6 		 Epoch time: 0m 6s
		 Training Loss: 0.31714092094350504 		 Validation Loss: 1.0989564565511851
Epoch 7 		 Epoch time: 0m 6s
		 Training Loss: 0.2813455330560336 		 Validation Loss: 1.0819417971831102
Epoch 8 		 Epoch time: 0m 6s
		 Training Loss: 0.24919866287225 		 Validation Loss: 1.049562014066256
Epoch 9 		 Epoch time: 0m 6s
		 Training Loss: 0.22254798903658585 		 Validation Loss: 1.0037054098

In [ ]:
for e in range(78):
  print(f'{e}. epoch')
  model.load_state_dict(torch.load(f'drive/MyDrive/complex/complex_models/{e}_model_Complex_1.pt'))

  predict_test_array = []

  with torch.no_grad():
      for i in range(len(test_X_num)):
          model.eval()
          predict = model(test_X_num[i].reshape(1,-1).to("cpu"), test_X_news[i].reshape(1,-1).to("cpu"))        
          predict_test_array.append(predict.item()*output_df.std()+output_df.mean()) 

  real_test_array = []
  real_trend_array = test["Trend target"]  
  for y in test_Y_num:
    real_test_array.append(y*output_df.std()+output_df.mean())

  # calculate score
  score = calc_score(predict_test_array,real_test_array)    
  print(f"\nScore: {score}\n")      

  #check the trend -> create an array with the real and with the predicted trend
  #if the current value is bigger than before -> 1
  #otherwise (same or smaller) -> 0
  predicted_trend_array = []
  for element in range(len(predict_test_array)):
      real_today_close = test["Close"][element]*input_df["Close"].std()+input_df["Close"].mean()
      if real_today_close > predict_test_array[element].values:
          predicted_trend_array.append(0)
      else:
          predicted_trend_array.append(1)

  #check the number of differences
  trend_diff_array = []
  for element in range(len(real_trend_array)):
      if real_trend_array[element] != predicted_trend_array[element]:
        trend_diff_array.append(element)

  #percentage of good predict
  acc = (len(real_trend_array)-len(trend_diff_array))/len(real_trend_array)*100
  print(f"Accuracy: {acc}\n")  
  #visualize it
  f2 = plt.figure(figsize=(24,12))
  plt.title("Predicted and real close prices", fontsize = 18)
  plt.plot(real_test_array, color = "blue", linewidth = 4,
          label = "Real")
  plt.plot(predict_test_array, color = "red", linewidth = 2,
          label = "Predicted")
  plt.xlabel("Date",fontsize = 18)
  plt.legend(fontsize = 18)
  f2.set_size_inches(12,6)
  plt_pred = f2
  plt.close()

  plt_pred.savefig('drive/MyDrive/complex/complex_models/'+str(e)+"_log_pred_" + str('Complex_1') + "_.png")

0. epoch

Score: 45628.812182741116

Accuracy: 48.984771573604064

1. epoch

Score: 47117.15736040609

Accuracy: 48.984771573604064

2. epoch

Score: 47998.64467005076

Accuracy: 48.984771573604064

3. epoch

Score: 48258.06598984772

Accuracy: 48.984771573604064

4. epoch

Score: 47869.8730964467

Accuracy: 48.984771573604064

5. epoch

Score: 46853.84263959391

Accuracy: 48.984771573604064

6. epoch

Score: 45243.131979695434

Accuracy: 48.984771573604064

7. epoch

Score: 43131.081218274114

Accuracy: 48.984771573604064

8. epoch

Score: 40610.37817258883

Accuracy: 48.984771573604064

9. epoch

Score: 37758.07614213198

Accuracy: 48.984771573604064

10. epoch

Score: 34698.82741116751

Accuracy: 48.984771573604064

11. epoch

Score: 31563.5

Accuracy: 48.984771573604064

12. epoch

Score: 28389.946700507615

Accuracy: 48.984771573604064

13. epoch

Score: 25269.42385786802

Accuracy: 48.984771573604064

14. epoch

Score: 22266.573604060915

Accuracy: 48.984771573604064

15. epoch



FileNotFoundError: ignored

In [ ]:
save_the_log(path="drive/MyDrive/complex/complex_models",name=model_name[0])

In [ ]:
model_name_tmp = ["Complex_1_LSTM_92_LSTM_news", "Complex_1_LSTM_36_LSTM_news"]
model_num = []
model_tmp = LSTM_92_num()
model_tmp.load_state_dict(torch.load('drive/MyDrive/complex/numerical_models/best_model_LSTM_92_num.pt'))
model_num.append(model_tmp)
model_tmp = LSTM_36_num()
model_tmp.load_state_dict(torch.load('drive/MyDrive/complex/numerical_models/best_model_LSTM_36_num.pt'))
model_num.append(model_tmp)
PAD_ID = vocab.lookup_indices(["<pad>"])[0]
model_news = LSTM_news(len(vocab),PAD_ID)
model_news.load_state_dict(torch.load(f'drive/MyDrive/complex/news_models/26_model_LSTM_pilot.pt'))

for i in range(2):
    model_name = []
    model_name.append(model_name_tmp[i])
    model_num_tmp = model_num[i]
    model = Complex_1(model_num_tmp, model_news)

    # freeze the numerical and news weights
    print(f'The model has {count_parameters(model):,} trainable parameters')

    for p in model.model_num.parameters():
        p.requires_grad = False
    for p in model.model_news.parameters():
        p.requires_grad = False
    print(f'The model has {count_parameters(model):,} trainable parameters')

    optimizer = optim.Adam(model.parameters(), 0.0001)
    criterion = nn.MSELoss()

    e_res = []
    acc_res = []
    batch_s_res = []
    optimizer_start_res = []
    optimizer_end_res = []
    dropout_res = []
    layer_dep_res = []
    act_res = []
    score_res = []
    plt_teach_res = []
    plt_pred_res = []

    epoch_out, score_out, acc_out, plt_teach_out, plt_best_out = run_model_complex(batch_size_in=32,model_in=model,
              optimizer_in=optimizer,criterion_in=criterion,model_name_in=model_name[0])

    e_res.append(epoch_out)
    acc_res.append(acc_out)
    batch_s_res.append("32")
    optimizer_start_res.append("0.0001")
    optimizer_end_res.append("0.0001")
    dropout_res.append("0")
    layer_dep_res.append("4")
    act_res.append("None")
    score_res.append(score_out)
    plt_teach_res.append(plt_teach_out)
    plt_pred_res.append(plt_best_out)

    save_the_log(path="drive/MyDrive/complex/complex_models",name=model_name[0])

The model has 2,320,883 trainable parameters
The model has 857 trainable parameters
Epoch 1 		 Epoch time: 0m 6s
		 Training Loss: 0.9182345874834101 		 Validation Loss: 0.4179050910931367
		 Validation Loss Decreased(inf--->0.417905) 	 Saving The Model
Epoch 2 		 Epoch time: 0m 6s
		 Training Loss: 0.7995279866044183 		 Validation Loss: 0.4770677788899495
Epoch 3 		 Epoch time: 0m 6s
		 Training Loss: 0.6903636807676505 		 Validation Loss: 0.5320911155297205
Epoch 4 		 Epoch time: 0m 6s
		 Training Loss: 0.5896841961940801 		 Validation Loss: 0.5814711485917752
Epoch 5 		 Epoch time: 0m 6s
		 Training Loss: 0.49745096961615254 		 Validation Loss: 0.6226301926832932
Epoch 6 		 Epoch time: 0m 6s
		 Training Loss: 0.4161853328775111 		 Validation Loss: 0.6540534289983603
Epoch 7 		 Epoch time: 0m 6s
		 Training Loss: 0.34384101580173987 		 Validation Loss: 0.6729669502148261
Epoch 8 		 Epoch time: 0m 6s
		 Training Loss: 0.28273807312487753 		 Validation Loss: 0.6779344517451066
Epoch 9 

In [ ]:
model_num = []
model_tmp = LSTM_92_num()
model_num.append(model_tmp)
model_tmp = LSTM_36_num()
model_num.append(model_tmp)

PAD_ID = vocab.lookup_indices(["<pad>"])[0]
model_news = LSTM_news(len(vocab),PAD_ID)

criterion = nn.MSELoss()

e = 75
for j in range(2):
  for k in range(e):
    print(f'{k}. epoch')
    if j == 0:
        name = "LSTM_92_LSTM_news"
        model_num_tmp = model_num[j]
        model = Complex_1(model_num_tmp, model_news)
    else:
        name = "LSTM_36_LSTM_news"
        model_num_tmp = model_num[j]
        model = Complex_1(model_num_tmp, model_news)

    model = model.to(device)
    model.load_state_dict(torch.load(f"drive/MyDrive/complex/complex_models/{k}_model_['Complex_1_{name}'].pt",map_location=torch.device('cpu')))

    predict_test_array = []

    with torch.no_grad():
        for i in range(len(test_X_num)):
            model.eval()
            predict = model(test_X_num[i].reshape(1,-1).to("cpu"), test_X_news[i].reshape(1,-1).to("cpu"))        
            predict_test_array.append(predict.item()*output_df.std()+output_df.mean()) 

    real_test_array = []
    real_trend_array = test["Trend target"]  
    for y in test_Y_num:
      real_test_array.append(y*output_df.std()+output_df.mean())

    # calculate score
    score = calc_score(predict_test_array,real_test_array)    
    print(f"\nScore: {score}\n")      

    #check the trend -> create an array with the real and with the predicted trend
    #if the current value is bigger than before -> 1
    #otherwise (same or smaller) -> 0
    predicted_trend_array = []
    for element in range(len(predict_test_array)):
        real_today_close = test["Close"][element]*input_df["Close"].std()+input_df["Close"].mean()
        if real_today_close > predict_test_array[element].values:
            predicted_trend_array.append(0)
        else:
            predicted_trend_array.append(1)

    #check the number of differences
    trend_diff_array = []
    for element in range(len(real_trend_array)):
        if real_trend_array[element] != predicted_trend_array[element]:
          trend_diff_array.append(element)

    #percentage of good predict
    acc = (len(real_trend_array)-len(trend_diff_array))/len(real_trend_array)*100
    print(f"Accuracy: {acc}\n")  
    #visualize it
    f2 = plt.figure(figsize=(24,12))
    plt.title("Predicted and real close prices", fontsize = 18)
    plt.plot(real_test_array, color = "blue", linewidth = 4,
            label = "Real")
    plt.plot(predict_test_array, color = "red", linewidth = 2,
            label = "Predicted")
    plt.xlabel("Date",fontsize = 18)
    plt.legend(fontsize = 18)
    f2.set_size_inches(12,6)
    plt_pred = f2
    plt.close()

    plt_pred.savefig('drive/MyDrive/complex/complex_models/'+str(k)+"_log_pred_" + str('Complex_1_') + str(name) + "_.png")

0. epoch

Score: 25574.644670050762

Accuracy: 48.984771573604064

1. epoch

Score: 27793.08375634518

Accuracy: 48.984771573604064

2. epoch

Score: 29663.294416243654

Accuracy: 48.984771573604064

3. epoch

Score: 31152.730964467006

Accuracy: 48.984771573604064

4. epoch

Score: 32177.598984771572

Accuracy: 48.984771573604064

5. epoch

Score: 32708.619289340102

Accuracy: 48.984771573604064

6. epoch

Score: 32668.479695431473

Accuracy: 48.984771573604064

7. epoch

Score: 32039.195431472082

Accuracy: 48.984771573604064

8. epoch

Score: 30820.58121827411

Accuracy: 48.984771573604064

9. epoch

Score: 29070.104060913705

Accuracy: 48.984771573604064

10. epoch

Score: 26898.040609137057

Accuracy: 48.984771573604064

11. epoch

Score: 24416.170050761422

Accuracy: 48.984771573604064

12. epoch

Score: 21762.096446700507

Accuracy: 48.984771573604064

13. epoch

Score: 19055.821065989847

Accuracy: 48.984771573604064

14. epoch

Score: 16413.10152284264

Accuracy: 48.9847715736

In [ ]:
# train with complex 2 and 3 the LSTM news
complex_name_tmp = ["Complex_2", "Complex_3"]
nums_name_tmp = ["Linear_108", "LSTM_92", "LSTM_36"]
news_name_tmp = ["LSTM_news"]

model_name_tmp = []
model_num = []
model_complex = []

PAD_ID = vocab.lookup_indices(["<pad>"])[0]
model_news = LSTM_news(len(vocab),PAD_ID)
model_news.load_state_dict(torch.load(f'drive/MyDrive/complex/news_models/26_model_LSTM_pilot.pt',map_location=torch.device('cpu')))

for complex_name_i in complex_name_tmp:
    for nums_name_i in nums_name_tmp:
        model_name_tmp.append(f"{complex_name_i}_{nums_name_i}_{news_name_tmp[0]}")

        if nums_name_i == "Linear_108":
          model_tmp = Linear_108_num()
          model_tmp.load_state_dict(torch.load('drive/MyDrive/complex/numerical_models/best_model_Linear_108.pt',map_location=torch.device('cpu')))
          model_num.append(model_tmp)
        elif nums_name_i == "LSTM_92":
          model_tmp = LSTM_92_num()
          model_tmp.load_state_dict(torch.load('drive/MyDrive/complex/numerical_models/best_model_LSTM_92_num.pt',map_location=torch.device('cpu')))
          model_num.append(model_tmp)
        elif nums_name_i == "LSTM_36":
          model_tmp = LSTM_36_num()
          model_tmp.load_state_dict(torch.load('drive/MyDrive/complex/numerical_models/best_model_LSTM_36_num.pt',map_location=torch.device('cpu')))
          model_num.append(model_tmp)        
        else:
          pass

        if complex_name_i == "Complex_2":
          model_tmp_complex = Complex_2(model_tmp, model_news)
          model_complex.append(model_tmp_complex)
        elif complex_name_i == "Complex_3":
          model_tmp_complex = Complex_3(model_tmp, model_news)
          model_complex.append(model_tmp_complex)
        else:
          pass   



for i in range(len(model_name_tmp)):
    model_name = []
    model_name.append(model_name_tmp[i])

    model = model_complex[i]

    # freeze the numerical and news weights
    print(f'The model has {count_parameters(model):,} trainable parameters')

    for p in model.model_num.parameters():
        p.requires_grad = False
    for p in model.model_news.parameters():
        p.requires_grad = False
    print(f'The model has {count_parameters(model):,} trainable parameters')

    optimizer = optim.Adam(model.parameters(), 0.0001)
    criterion = nn.MSELoss()

    e_res = []
    acc_res = []
    batch_s_res = []
    optimizer_start_res = []
    optimizer_end_res = []
    dropout_res = []
    layer_dep_res = []
    act_res = []
    score_res = []
    plt_teach_res = []
    plt_pred_res = []

    epoch_out, score_out, acc_out, plt_teach_out, plt_best_out = run_model_complex(batch_size_in=32,model_in=model,
              optimizer_in=optimizer,criterion_in=criterion,model_name_in=model_name[0])

    e_res.append(epoch_out)
    acc_res.append(acc_out)
    batch_s_res.append("32")
    optimizer_start_res.append("0.0001")
    optimizer_end_res.append("0.0001")
    dropout_res.append("TBD")
    layer_dep_res.append("TBD")
    act_res.append("TBD")
    score_res.append(score_out)
    plt_teach_res.append(plt_teach_out)
    plt_pred_res.append(plt_best_out)

    save_the_log(path="drive/MyDrive/complex/complex_models",name=model_name[0])

The model has 1,558,035 trainable parameters
The model has 3,881 trainable parameters
Epoch 1 		 Epoch time: 0m 8s
		 Training Loss: 0.7777004946665084 		 Validation Loss: 0.9232713671830984
		 Validation Loss Decreased(inf--->0.923271) 	 Saving The Model
Epoch 2 		 Epoch time: 0m 8s
		 Training Loss: 0.5972942649386823 		 Validation Loss: 1.107208673770611
Epoch 3 		 Epoch time: 0m 8s
		 Training Loss: 0.4517491597782921 		 Validation Loss: 1.2916871355130122
Epoch 4 		 Epoch time: 0m 8s
		 Training Loss: 0.346736437779524 		 Validation Loss: 1.418138311459468
Epoch 5 		 Epoch time: 0m 8s
		 Training Loss: 0.2834654058380103 		 Validation Loss: 1.4520949813035817
Epoch 6 		 Epoch time: 0m 8s
		 Training Loss: 0.2453059133998043 		 Validation Loss: 1.4119866765462434
Epoch 7 		 Epoch time: 0m 8s
		 Training Loss: 0.22140789150285559 		 Validation Loss: 1.3343511407191937
Epoch 8 		 Epoch time: 0m 8s
		 Training Loss: 0.2015156445496187 		 Validation Loss: 1.2397804856300354
Epoch 9 		 

In [ ]:
# check the complex results epoch to epoch
criterion = nn.MSELoss()

e = [100,199,101,198,139,199]
for j in range(len(model_name_tmp)):
  for k in range(e[j]):
    print(f'{k}. epoch')

    name = model_name_tmp[j]
    model = model_complex[j]

    model = model.to(device)
    model.load_state_dict(torch.load(f"drive/MyDrive/complex/complex_models/{k}_model_{name}.pt",map_location=torch.device('cpu')))

    predict_test_array = []

    with torch.no_grad():
        for i in range(len(test_X_num)):
            model.eval()
            predict = model(test_X_num[i].reshape(1,-1).to("cpu"), test_X_news[i].reshape(1,-1).to("cpu"))        
            predict_test_array.append(predict.item()*output_df.std()+output_df.mean()) 

    real_test_array = []
    real_trend_array = test["Trend target"]  
    for y in test_Y_num:
      real_test_array.append(y*output_df.std()+output_df.mean())

    # calculate score
    score = calc_score(predict_test_array,real_test_array)    
    print(f"\nScore: {score}\n")      

    #check the trend -> create an array with the real and with the predicted trend
    #if the current value is bigger than before -> 1
    #otherwise (same or smaller) -> 0
    predicted_trend_array = []
    for element in range(len(predict_test_array)):
        real_today_close = test["Close"][element]*input_df["Close"].std()+input_df["Close"].mean()
        if real_today_close > predict_test_array[element].values:
            predicted_trend_array.append(0)
        else:
            predicted_trend_array.append(1)

    #check the number of differences
    trend_diff_array = []
    for element in range(len(real_trend_array)):
        if real_trend_array[element] != predicted_trend_array[element]:
          trend_diff_array.append(element)

    #percentage of good predict
    acc = (len(real_trend_array)-len(trend_diff_array))/len(real_trend_array)*100
    print(f"Accuracy: {acc}\n")  
    #visualize it
    f2 = plt.figure(figsize=(24,12))
    plt.title("Predicted and real close prices", fontsize = 18)
    plt.plot(real_test_array, color = "blue", linewidth = 4,
            label = "Real")
    plt.plot(predict_test_array, color = "red", linewidth = 2,
            label = "Predicted")
    plt.xlabel("Date",fontsize = 18)
    plt.legend(fontsize = 18)
    f2.set_size_inches(12,6)
    plt_pred = f2
    plt.close()

    plt_pred.savefig('drive/MyDrive/complex/complex_models/'+str(k)+"_log_pred_" + str(name) + "_.png")

0. epoch

Score: 47233.82233502538

Accuracy: 48.984771573604064

1. epoch

Score: 53255.97969543147

Accuracy: 48.984771573604064

2. epoch

Score: 58794.72588832487

Accuracy: 48.984771573604064

3. epoch

Score: 61972.26395939086

Accuracy: 48.984771573604064

4. epoch

Score: 61940.46192893401

Accuracy: 48.984771573604064

5. epoch

Score: 59525.17258883249

Accuracy: 48.984771573604064

6. epoch

Score: 55945.47715736041

Accuracy: 48.984771573604064

7. epoch

Score: 51849.857868020306

Accuracy: 48.984771573604064

8. epoch

Score: 47470.137055837564

Accuracy: 48.984771573604064

9. epoch

Score: 42965.3807106599

Accuracy: 48.984771573604064

10. epoch

Score: 38322.71319796954

Accuracy: 48.984771573604064

11. epoch

Score: 33574.37055837563

Accuracy: 48.984771573604064

12. epoch

Score: 28773.327411167513

Accuracy: 48.984771573604064

13. epoch

Score: 23991.53299492386

Accuracy: 48.984771573604064

14. epoch

Score: 19337.1230964467

Accuracy: 48.984771573604064

15. 

In [ ]:
# check the complex results epoch to epoch
complex_name_tmp = ["Complex_2", "Complex_3"]
nums_name_tmp = ["Linear_108", "LSTM_92", "LSTM_36"]
news_name_tmp = ["LSTM_news"]

model_name_tmp = []
model_num = []
model_complex = []

PAD_ID = vocab.lookup_indices(["<pad>"])[0]
model_news = LSTM_news(len(vocab),PAD_ID)
model_news.load_state_dict(torch.load(f'drive/MyDrive/complex/news_models/26_model_LSTM_pilot.pt',map_location=torch.device('cpu')))

for complex_name_i in complex_name_tmp:
    for nums_name_i in nums_name_tmp:
        model_name_tmp.append(f"{complex_name_i}_{nums_name_i}_{news_name_tmp[0]}")

        if nums_name_i == "Linear_108":
          model_tmp = Linear_108_num()
          model_tmp.load_state_dict(torch.load('drive/MyDrive/complex/numerical_models/best_model_Linear_108.pt',map_location=torch.device('cpu')))
          model_num.append(model_tmp)
        elif nums_name_i == "LSTM_92":
          model_tmp = LSTM_92_num()
          model_tmp.load_state_dict(torch.load('drive/MyDrive/complex/numerical_models/best_model_LSTM_92_num.pt',map_location=torch.device('cpu')))
          model_num.append(model_tmp)
        elif nums_name_i == "LSTM_36":
          model_tmp = LSTM_36_num()
          model_tmp.load_state_dict(torch.load('drive/MyDrive/complex/numerical_models/best_model_LSTM_36_num.pt',map_location=torch.device('cpu')))
          model_num.append(model_tmp)        
        else:
          pass

        if complex_name_i == "Complex_2":
          model_tmp_complex = Complex_2(model_tmp, model_news)
          model_complex.append(model_tmp_complex)
        elif complex_name_i == "Complex_3":
          model_tmp_complex = Complex_3(model_tmp, model_news)
          model_complex.append(model_tmp_complex)
        else:
          pass   

criterion = nn.MSELoss()

e = [100,199,101,198,139,199]
for j in range(1,len(model_name_tmp)):
  if j == 1:
      for k in range(174,e[j]):
        print(f'{k}. epoch')

        name = model_name_tmp[j]
        model = model_complex[j]

        model = model.to(device)
        model.load_state_dict(torch.load(f"drive/MyDrive/complex/complex_models/{k}_model_{name}.pt",map_location=torch.device('cpu')))

        predict_test_array = []

        with torch.no_grad():
            for i in range(len(test_X_num)):
                model.eval()
                predict = model(test_X_num[i].reshape(1,-1).to("cpu"), test_X_news[i].reshape(1,-1).to("cpu"))        
                predict_test_array.append(predict.item()*output_df.std()+output_df.mean()) 

        real_test_array = []
        real_trend_array = test["Trend target"]  
        for y in test_Y_num:
          real_test_array.append(y*output_df.std()+output_df.mean())

        # calculate score
        score = calc_score(predict_test_array,real_test_array)    
        print(f"\nScore: {score}\n")      

        #check the trend -> create an array with the real and with the predicted trend
        #if the current value is bigger than before -> 1
        #otherwise (same or smaller) -> 0
        predicted_trend_array = []
        for element in range(len(predict_test_array)):
            real_today_close = test["Close"][element]*input_df["Close"].std()+input_df["Close"].mean()
            if real_today_close > predict_test_array[element].values:
                predicted_trend_array.append(0)
            else:
                predicted_trend_array.append(1)

        #check the number of differences
        trend_diff_array = []
        for element in range(len(real_trend_array)):
            if real_trend_array[element] != predicted_trend_array[element]:
              trend_diff_array.append(element)

        #percentage of good predict
        acc = (len(real_trend_array)-len(trend_diff_array))/len(real_trend_array)*100
        print(f"Accuracy: {acc}\n")  
        #visualize it
        f2 = plt.figure(figsize=(24,12))
        plt.title("Predicted and real close prices", fontsize = 18)
        plt.plot(real_test_array, color = "blue", linewidth = 4,
                label = "Real")
        plt.plot(predict_test_array, color = "red", linewidth = 2,
                label = "Predicted")
        plt.xlabel("Date",fontsize = 18)
        plt.legend(fontsize = 18)
        f2.set_size_inches(12,6)
        plt_pred = f2
        plt.close()

        plt_pred.savefig('drive/MyDrive/complex/complex_models/'+str(k)+"_log_pred_" + str(name) + "_.png")
  else:
      for k in range(e[j]):
        print(f'{k}. epoch')

        name = model_name_tmp[j]
        model = model_complex[j]

        model = model.to(device)
        model.load_state_dict(torch.load(f"drive/MyDrive/complex/complex_models/{k}_model_{name}.pt",map_location=torch.device('cpu')))

        predict_test_array = []

        with torch.no_grad():
            for i in range(len(test_X_num)):
                model.eval()
                predict = model(test_X_num[i].reshape(1,-1).to("cpu"), test_X_news[i].reshape(1,-1).to("cpu"))        
                predict_test_array.append(predict.item()*output_df.std()+output_df.mean()) 

        real_test_array = []
        real_trend_array = test["Trend target"]  
        for y in test_Y_num:
          real_test_array.append(y*output_df.std()+output_df.mean())

        # calculate score
        score = calc_score(predict_test_array,real_test_array)    
        print(f"\nScore: {score}\n")      

        #check the trend -> create an array with the real and with the predicted trend
        #if the current value is bigger than before -> 1
        #otherwise (same or smaller) -> 0
        predicted_trend_array = []
        for element in range(len(predict_test_array)):
            real_today_close = test["Close"][element]*input_df["Close"].std()+input_df["Close"].mean()
            if real_today_close > predict_test_array[element].values:
                predicted_trend_array.append(0)
            else:
                predicted_trend_array.append(1)

        #check the number of differences
        trend_diff_array = []
        for element in range(len(real_trend_array)):
            if real_trend_array[element] != predicted_trend_array[element]:
              trend_diff_array.append(element)

        #percentage of good predict
        acc = (len(real_trend_array)-len(trend_diff_array))/len(real_trend_array)*100
        print(f"Accuracy: {acc}\n")  
        #visualize it
        f2 = plt.figure(figsize=(24,12))
        plt.title("Predicted and real close prices", fontsize = 18)
        plt.plot(real_test_array, color = "blue", linewidth = 4,
                label = "Real")
        plt.plot(predict_test_array, color = "red", linewidth = 2,
                label = "Predicted")
        plt.xlabel("Date",fontsize = 18)
        plt.legend(fontsize = 18)
        f2.set_size_inches(12,6)
        plt_pred = f2
        plt.close()

        plt_pred.savefig('drive/MyDrive/complex/complex_models/'+str(k)+"_log_pred_" + str(name) + "_.png")

174. epoch

Score: 126.62183732550761

Accuracy: 50.76142131979695

175. epoch

Score: 108.59615918464468

Accuracy: 52.03045685279187

176. epoch

Score: 129.5793742068528

Accuracy: 50.25380710659898

177. epoch

Score: 108.50812975888324

Accuracy: 52.53807106598985

178. epoch

Score: 114.12445471129442

Accuracy: 49.746192893401016

179. epoch

Score: 109.57637016180203

Accuracy: 52.28426395939086

180. epoch

Score: 110.03932027284264

Accuracy: 52.03045685279187

181. epoch

Score: 107.94359731916244

Accuracy: 53.045685279187815

182. epoch

Score: 115.9270800285533

Accuracy: 49.746192893401016

183. epoch

Score: 116.12994725571066

Accuracy: 50.25380710659898

184. epoch

Score: 115.70664459073605

Accuracy: 50.0

185. epoch

Score: 120.6816505393401

Accuracy: 50.0

186. epoch

Score: 116.84865759835026

Accuracy: 49.746192893401016

187. epoch

Score: 119.95931154822335

Accuracy: 50.0

188. epoch

Score: 127.63180123730965

Accuracy: 50.50761421319797

189. epoch

Score:

In [ ]:
# train all complex with logreg -> there the check may be not epoch to epoch
complex_name_tmp = ["Complex_1", "Complex_2", "Complex_3"]
nums_name_tmp = ["Linear_108", "LSTM_92", "LSTM_36"]
news_name_tmp = ["LogReg"]

model_name_tmp = []
model_num = []
model_complex = []

model_news = pickle.load(open('drive/MyDrive/complex/news_models/best_model_LogReg.pt', 'rb'))


for complex_name_i in complex_name_tmp:
    for nums_name_i in nums_name_tmp:
        model_name_tmp.append(f"{complex_name_i}_{nums_name_i}_{news_name_tmp[0]}")

        if nums_name_i == "Linear_108":
          model_tmp = Linear_108_num()
          model_tmp.load_state_dict(torch.load('drive/MyDrive/complex/numerical_models/best_model_Linear_108.pt', map_location=torch.device('cpu')))
          model_num.append(model_tmp)
        elif nums_name_i == "LSTM_92":
          model_tmp = LSTM_92_num()
          model_tmp.load_state_dict(torch.load('drive/MyDrive/complex/numerical_models/best_model_LSTM_92_num.pt', map_location=torch.device('cpu')))
          model_num.append(model_tmp)
        elif nums_name_i == "LSTM_36":
          model_tmp = LSTM_36_num()
          model_tmp.load_state_dict(torch.load('drive/MyDrive/complex/numerical_models/best_model_LSTM_36_num.pt', map_location=torch.device('cpu')))
          model_num.append(model_tmp)        
        else:
          pass

        if complex_name_i == "Complex_1":
          model_tmp_complex = Complex_1(model_tmp, model_news)
          model_complex.append(model_tmp_complex)
        elif complex_name_i == "Complex_2":
          model_tmp_complex = Complex_2(model_tmp, model_news)
          model_complex.append(model_tmp_complex)          
        elif complex_name_i == "Complex_3":
          model_tmp_complex = Complex_3(model_tmp, model_news)
          model_complex.append(model_tmp_complex)
        else:
          pass   



for i in range(len(model_name_tmp)):
    model_name = []
    model_name.append(model_name_tmp[i])

    model = model_complex[i]

    print(f"\n{model_name[0]}\n")

    # freeze the numerical and news weights
    print(f'The model has {count_parameters(model):,} trainable parameters')

    for p in model.model_num.parameters():
        p.requires_grad = False
    print(f'The model has {count_parameters(model):,} trainable parameters')

    optimizer = optim.Adam(model.parameters(), 0.001)
    criterion = nn.MSELoss()

    e_res = []
    acc_res = []
    batch_s_res = []
    optimizer_start_res = []
    optimizer_end_res = []
    dropout_res = []
    layer_dep_res = []
    act_res = []
    score_res = []
    plt_teach_res = []
    plt_pred_res = []

    epoch_out, score_out, acc_out, plt_teach_out, plt_best_out = run_model_complex(batch_size_in=32,model_in=model,
              optimizer_in=optimizer,criterion_in=criterion,model_name_in=model_name[0])

    e_res.append(epoch_out)
    acc_res.append(acc_out)
    batch_s_res.append("32")
    optimizer_start_res.append("001")
    optimizer_end_res.append("001")
    dropout_res.append("TBD")
    layer_dep_res.append("TBD")
    act_res.append("TBD")
    score_res.append(score_out)
    plt_teach_res.append(plt_teach_out)
    plt_pred_res.append(plt_best_out)

    save_the_log(path="drive/MyDrive/complex/complex_models",name=model_name[0])


Complex_1_Linear_108_LogReg

The model has 1,398,266 trainable parameters
The model has 857 trainable parameters
Epoch 1 		 Epoch time: 0m 2s
		 Training Loss: 0.5637512532742442 		 Validation Loss: 1.3721302976975074
		 Validation Loss Decreased(inf--->1.372130) 	 Saving The Model
Epoch 2 		 Epoch time: 0m 2s
		 Training Loss: 0.2229420849550012 		 Validation Loss: 0.8375803255117856
		 Validation Loss Decreased(1.372130--->0.837580) 	 Saving The Model
Epoch 3 		 Epoch time: 0m 2s
		 Training Loss: 0.14262602732491655 		 Validation Loss: 0.3590233429120137
		 Validation Loss Decreased(0.837580--->0.359023) 	 Saving The Model
Epoch 4 		 Epoch time: 0m 2s
		 Training Loss: 0.05740112837127133 		 Validation Loss: 0.05410323435297379
		 Validation Loss Decreased(0.359023--->0.054103) 	 Saving The Model
Epoch 5 		 Epoch time: 0m 2s
		 Training Loss: 0.010826008652362067 		 Validation Loss: 0.0022393836071177456
		 Validation Loss Decreased(0.054103--->0.002239) 	 Saving The Model
Epoch 6 

In [ ]:
# train all complex with VADER -> there the check may be not epoch to epoch
complex_name_tmp = ["Complex_1", "Complex_2", "Complex_3"]
nums_name_tmp = ["Linear_108", "LSTM_92", "LSTM_36"]
news_name_tmp = ["VADER"]

model_name_tmp = []
model_num = []
model_complex = []

model_news = VADER()

for complex_name_i in complex_name_tmp:
    for nums_name_i in nums_name_tmp:
        model_name_tmp.append(f"{complex_name_i}_{nums_name_i}_{news_name_tmp[0]}")

        if nums_name_i == "Linear_108":
          model_tmp = Linear_108_num()
          model_tmp.load_state_dict(torch.load('drive/MyDrive/complex/numerical_models/best_model_Linear_108.pt', map_location=torch.device('cpu')))
          model_num.append(model_tmp)
        elif nums_name_i == "LSTM_92":
          model_tmp = LSTM_92_num()
          model_tmp.load_state_dict(torch.load('drive/MyDrive/complex/numerical_models/best_model_LSTM_92_num.pt', map_location=torch.device('cpu')))
          model_num.append(model_tmp)
        elif nums_name_i == "LSTM_36":
          model_tmp = LSTM_36_num()
          model_tmp.load_state_dict(torch.load('drive/MyDrive/complex/numerical_models/best_model_LSTM_36_num.pt', map_location=torch.device('cpu')))
          model_num.append(model_tmp)        
        else:
          pass

        if complex_name_i == "Complex_1":
          model_tmp_complex = Complex_1(model_tmp, model_news)
          model_complex.append(model_tmp_complex)
        elif complex_name_i == "Complex_2":
          model_tmp_complex = Complex_2(model_tmp, model_news)
          model_complex.append(model_tmp_complex)          
        elif complex_name_i == "Complex_3":
          model_tmp_complex = Complex_3(model_tmp, model_news)
          model_complex.append(model_tmp_complex)
        else:
          pass   



for i in range(len(model_name_tmp)):
    model_name = []
    model_name.append(model_name_tmp[i])

    model = model_complex[i]

    print(f"\n{model_name[0]}\n")

    # freeze the numerical and news weights
    print(f'The model has {count_parameters(model):,} trainable parameters')

    for p in model.model_num.parameters():
        p.requires_grad = False
    print(f'The model has {count_parameters(model):,} trainable parameters')

    optimizer = optim.Adam(model.parameters(), 0.0001)
    criterion = nn.MSELoss()

    e_res = []
    acc_res = []
    batch_s_res = []
    optimizer_start_res = []
    optimizer_end_res = []
    dropout_res = []
    layer_dep_res = []
    act_res = []
    score_res = []
    plt_teach_res = []
    plt_pred_res = []

    epoch_out, score_out, acc_out, plt_teach_out, plt_best_out = run_model_complex(batch_size_in=32,model_in=model,
              optimizer_in=optimizer,criterion_in=criterion,model_name_in=model_name[0])

    e_res.append(epoch_out)
    acc_res.append(acc_out)
    batch_s_res.append("32")
    optimizer_start_res.append("0.0001")
    optimizer_end_res.append("0.0001")
    dropout_res.append("TBD")
    layer_dep_res.append("TBD")
    act_res.append("TBD")
    score_res.append(score_out)
    plt_teach_res.append(plt_teach_out)
    plt_pred_res.append(plt_best_out)

    save_the_log(path="drive/MyDrive/complex/complex_models",name=model_name[0])


Complex_1_Linear_108_VADER

The model has 1,398,266 trainable parameters
The model has 857 trainable parameters
Epoch 1 		 Epoch time: 0m 0s
		 Training Loss: 0.8205133659161024 		 Validation Loss: 0.7502201039057511
		 Validation Loss Decreased(inf--->0.750220) 	 Saving The Model
Epoch 2 		 Epoch time: 0m 0s
		 Training Loss: 0.7022544966721153 		 Validation Loss: 0.8109293740529281
Epoch 3 		 Epoch time: 0m 0s
		 Training Loss: 0.6017229962917799 		 Validation Loss: 0.8636668714193197
Epoch 4 		 Epoch time: 0m 0s
		 Training Loss: 0.5130690282613442 		 Validation Loss: 0.9081500952060406
Epoch 5 		 Epoch time: 0m 0s
		 Training Loss: 0.43547707647588607 		 Validation Loss: 0.9441705048084259
Epoch 6 		 Epoch time: 0m 0s
		 Training Loss: 0.36838063059022297 		 Validation Loss: 0.9710197586279649
Epoch 7 		 Epoch time: 0m 0s
		 Training Loss: 0.31145669346851473 		 Validation Loss: 0.9875978598227868
Epoch 8 		 Epoch time: 0m 0s
		 Training Loss: 0.26427223741051714 		 Validation Los

In [ ]:
# train all complex with BERT -> need to rerun the preprocess -> there the check may be not epoch to epoch
complex_name_tmp = ["Complex_1", "Complex_2", "Complex_3"]
nums_name_tmp = ["Linear_108", "LSTM_92", "LSTM_36"]
news_name_tmp = ["BERT_news"]

model_name_tmp = []
model_num = []
model_complex = []

model_news = BERT_news()
model_news.load_state_dict(torch.load(f'drive/MyDrive/complex/news_models/71_model_BERT.pt'))

for complex_name_i in complex_name_tmp:
    for nums_name_i in nums_name_tmp:
        model_name_tmp.append(f"{complex_name_i}_{nums_name_i}_{news_name_tmp[0]}")

        if nums_name_i == "Linear_108":
          model_tmp = Linear_108_num()
          model_tmp.load_state_dict(torch.load('drive/MyDrive/complex/numerical_models/best_model_Linear_108.pt'))
          model_num.append(model_tmp)
        elif nums_name_i == "LSTM_92":
          model_tmp = LSTM_92_num()
          model_tmp.load_state_dict(torch.load('drive/MyDrive/complex/numerical_models/best_model_LSTM_92_num.pt'))
          model_num.append(model_tmp)
        elif nums_name_i == "LSTM_36":
          model_tmp = LSTM_36_num()
          model_tmp.load_state_dict(torch.load('drive/MyDrive/complex/numerical_models/best_model_LSTM_36_num.pt'))
          model_num.append(model_tmp)        
        else:
          pass

        if complex_name_i == "Complex_1":
          model_tmp_complex = Complex_1(model_tmp, model_news)
          model_complex.append(model_tmp_complex)
        elif complex_name_i == "Complex_2":
          model_tmp_complex = Complex_2(model_tmp, model_news)
          model_complex.append(model_tmp_complex)          
        elif complex_name_i == "Complex_3":
          model_tmp_complex = Complex_3(model_tmp, model_news)
          model_complex.append(model_tmp_complex)
        else:
          pass   



for i in range(len(model_name_tmp)):
    model_name = []
    model_name.append(model_name_tmp[i])

    model = model_complex[i]

    print(f"\n{model_name}\n")

    # freeze the numerical and news weights
    print(f'The model has {count_parameters(model):,} trainable parameters')

    for p in model.model_num.parameters():
        p.requires_grad = False
    for p in model.model_news.parameters():
        p.requires_grad = False
    print(f'The model has {count_parameters(model):,} trainable parameters')

    optimizer = optim.Adam(model.parameters(), 0.0001)
    criterion = nn.MSELoss()

    e_res = []
    acc_res = []
    batch_s_res = []
    optimizer_start_res = []
    optimizer_end_res = []
    dropout_res = []
    layer_dep_res = []
    act_res = []
    score_res = []
    plt_teach_res = []
    plt_pred_res = []

    epoch_out, score_out, acc_out, plt_teach_out, plt_best_out = run_model_complex(batch_size_in=32,model_in=model,
              optimizer_in=optimizer,criterion_in=criterion,model_name_in=model_name[0])

    e_res.append(epoch_out)
    acc_res.append(acc_out)
    batch_s_res.append("32")
    optimizer_start_res.append("0.0001")
    optimizer_end_res.append("0.0001")
    dropout_res.append("TBD")
    layer_dep_res.append("TBD")
    act_res.append("TBD")
    score_res.append(score_out)
    plt_teach_res.append(plt_teach_out)
    plt_pred_res.append(plt_best_out)

    save_the_log(path="drive/MyDrive/complex/complex_models",name=model_name[0]) # todo: need to rerun / continue from 2

The model has 1,481,627 trainable parameters
The model has 857 trainable parameters
Epoch 1 		 Epoch time: 1m 48s
		 Training Loss: 0.5871091664366022 		 Validation Loss: 1.4165715345969567
		 Validation Loss Decreased(inf--->1.416572) 	 Saving The Model
Epoch 2 		 Epoch time: 1m 48s
		 Training Loss: 0.5281120884685299 		 Validation Loss: 1.4837551712989807
Epoch 3 		 Epoch time: 1m 48s
		 Training Loss: 0.480121005855098 		 Validation Loss: 1.5314376812714796
Epoch 4 		 Epoch time: 1m 48s
		 Training Loss: 0.4380995084065944 		 Validation Loss: 1.5600238304871779
Epoch 5 		 Epoch time: 1m 48s
		 Training Loss: 0.4012207860706022 		 Validation Loss: 1.5709874538274913
Epoch 6 		 Epoch time: 1m 48s
		 Training Loss: 0.3681046791827759 		 Validation Loss: 1.5644356195743268
Epoch 7 		 Epoch time: 1m 48s
		 Training Loss: 0.33885162484806935 		 Validation Loss: 1.5418700850926912
Epoch 8 		 Epoch time: 1m 48s
		 Training Loss: 0.3128637117837128 		 Validation Loss: 1.5050052083455598
Epo

In [ ]:
# train all complex with BERT -> need to rerun the preprocess -> there the check may be not epoch to epoch
complex_name_tmp = ["Complex_1", "Complex_2", "Complex_3"]
nums_name_tmp = ["Linear_108", "LSTM_92", "LSTM_36"]
news_name_tmp = ["BERT_news"]

model_name_tmp = []
model_num = []
model_complex = []

model_news = BERT_news()
model_news.load_state_dict(torch.load(f'drive/MyDrive/complex/news_models/71_model_BERT.pt'))

for complex_name_i in complex_name_tmp:
    for nums_name_i in nums_name_tmp:
        model_name_tmp.append(f"{complex_name_i}_{nums_name_i}_{news_name_tmp[0]}")

        if nums_name_i == "Linear_108":
          model_tmp = Linear_108_num()
          model_tmp.load_state_dict(torch.load('drive/MyDrive/complex/numerical_models/best_model_Linear_108.pt'))
          model_num.append(model_tmp)
        elif nums_name_i == "LSTM_92":
          model_tmp = LSTM_92_num()
          model_tmp.load_state_dict(torch.load('drive/MyDrive/complex/numerical_models/best_model_LSTM_92_num.pt'))
          model_num.append(model_tmp)
        elif nums_name_i == "LSTM_36":
          model_tmp = LSTM_36_num()
          model_tmp.load_state_dict(torch.load('drive/MyDrive/complex/numerical_models/best_model_LSTM_36_num.pt'))
          model_num.append(model_tmp)        
        else:
          pass

        if complex_name_i == "Complex_1":
          model_tmp_complex = Complex_1(model_tmp, model_news)
          model_complex.append(model_tmp_complex)
        elif complex_name_i == "Complex_2":
          model_tmp_complex = Complex_2(model_tmp, model_news)
          model_complex.append(model_tmp_complex)          
        elif complex_name_i == "Complex_3":
          model_tmp_complex = Complex_3(model_tmp, model_news)
          model_complex.append(model_tmp_complex)
        else:
          pass   



for i in range(1, len(model_name_tmp)):
    model_name = []
    model_name.append(model_name_tmp[i])

    model = model_complex[i]

    print(f"{model_name[0]}\n")

    # freeze the numerical and news weights
    print(f'The model has {count_parameters(model):,} trainable parameters')

    for p in model.model_num.parameters():
        p.requires_grad = False
    for p in model.model_news.parameters():
        p.requires_grad = False
    print(f'The model has {count_parameters(model):,} trainable parameters')

    optimizer = optim.Adam(model.parameters(), 0.0001)
    criterion = nn.MSELoss()

    e_res = []
    acc_res = []
    batch_s_res = []
    optimizer_start_res = []
    optimizer_end_res = []
    dropout_res = []
    layer_dep_res = []
    act_res = []
    score_res = []
    plt_teach_res = []
    plt_pred_res = []

    epoch_out, score_out, acc_out, plt_teach_out, plt_best_out = run_model_complex(batch_size_in=32,model_in=model,
              optimizer_in=optimizer,criterion_in=criterion,model_name_in=model_name[0])

    e_res.append(epoch_out)
    acc_res.append(acc_out)
    batch_s_res.append("32")
    optimizer_start_res.append("0.0001")
    optimizer_end_res.append("0.0001")
    dropout_res.append("TBD")
    layer_dep_res.append("TBD")
    act_res.append("TBD")
    score_res.append(score_out)
    plt_teach_res.append(plt_teach_out)
    plt_pred_res.append(plt_best_out)

    save_the_log(path="drive/MyDrive/complex/complex_models",name=model_name[0])

Complex_1_LSTM_92_BERT_news

The model has 111,729,739 trainable parameters
The model has 857 trainable parameters
Epoch 1 		 Epoch time: 1m 45s
		 Training Loss: 0.9408257897515353 		 Validation Loss: 0.8650538394084344
		 Validation Loss Decreased(inf--->0.865054) 	 Saving The Model
Epoch 2 		 Epoch time: 1m 46s
		 Training Loss: 0.809583116026998 		 Validation Loss: 0.9217331959651067
Epoch 3 		 Epoch time: 1m 46s
		 Training Loss: 0.6935437537629057 		 Validation Loss: 0.9746408393749824
Epoch 4 		 Epoch time: 1m 46s
		 Training Loss: 0.5884939457944317 		 Validation Loss: 1.0204669695634108
Epoch 5 		 Epoch time: 1m 46s
		 Training Loss: 0.4958254685751288 		 Validation Loss: 1.0570586507137005
Epoch 6 		 Epoch time: 1m 46s
		 Training Loss: 0.4157972204045871 		 Validation Loss: 1.0829536135380085
Epoch 7 		 Epoch time: 1m 46s
		 Training Loss: 0.3501531849615276 		 Validation Loss: 1.0928952143742487
Epoch 8 		 Epoch time: 1m 46s
		 Training Loss: 0.2963723861486525 		 Validatio

In [ ]:
# train all complex with BERT -> need to rerun the preprocess -> there the check may be not epoch to epoch
complex_name_tmp = ["Complex_1", "Complex_2", "Complex_3"]
nums_name_tmp = ["Linear_108", "LSTM_92", "LSTM_36"]
news_name_tmp = ["BERT_news"]

model_name_tmp = []
model_num = []
model_complex = []

model_news = BERT_news()
model_news.load_state_dict(torch.load(f'drive/MyDrive/complex/news_models/71_model_BERT.pt'))

for complex_name_i in complex_name_tmp:
    for nums_name_i in nums_name_tmp:
        model_name_tmp.append(f"{complex_name_i}_{nums_name_i}_{news_name_tmp[0]}")

        if nums_name_i == "Linear_108":
          model_tmp = Linear_108_num()
          model_tmp.load_state_dict(torch.load('drive/MyDrive/complex/numerical_models/best_model_Linear_108.pt'))
          model_num.append(model_tmp)
        elif nums_name_i == "LSTM_92":
          model_tmp = LSTM_92_num()
          model_tmp.load_state_dict(torch.load('drive/MyDrive/complex/numerical_models/best_model_LSTM_92_num.pt'))
          model_num.append(model_tmp)
        elif nums_name_i == "LSTM_36":
          model_tmp = LSTM_36_num()
          model_tmp.load_state_dict(torch.load('drive/MyDrive/complex/numerical_models/best_model_LSTM_36_num.pt'))
          model_num.append(model_tmp)        
        else:
          pass

        if complex_name_i == "Complex_1":
          model_tmp_complex = Complex_1(model_tmp, model_news)
          model_complex.append(model_tmp_complex)
        elif complex_name_i == "Complex_2":
          model_tmp_complex = Complex_2(model_tmp, model_news)
          model_complex.append(model_tmp_complex)          
        elif complex_name_i == "Complex_3":
          model_tmp_complex = Complex_3(model_tmp, model_news)
          model_complex.append(model_tmp_complex)
        else:
          pass   



for i in range(4, len(model_name_tmp)):
    model_name = []
    model_name.append(model_name_tmp[i])

    model = model_complex[i]

    print(f"{model_name[0]}\n")

    # freeze the numerical and news weights
    print(f'The model has {count_parameters(model):,} trainable parameters')

    for p in model.model_num.parameters():
        p.requires_grad = False
    for p in model.model_news.parameters():
        p.requires_grad = False
    print(f'The model has {count_parameters(model):,} trainable parameters')

    optimizer = optim.Adam(model.parameters(), 0.0001)
    criterion = nn.MSELoss()

    e_res = []
    acc_res = []
    batch_s_res = []
    optimizer_start_res = []
    optimizer_end_res = []
    dropout_res = []
    layer_dep_res = []
    act_res = []
    score_res = []
    plt_teach_res = []
    plt_pred_res = []

    epoch_out, score_out, acc_out, plt_teach_out, plt_best_out = run_model_complex(batch_size_in=32,model_in=model,
              optimizer_in=optimizer,criterion_in=criterion,model_name_in=model_name[0])

    e_res.append(epoch_out)
    acc_res.append(acc_out)
    batch_s_res.append("32")
    optimizer_start_res.append("0.0001")
    optimizer_end_res.append("0.0001")
    dropout_res.append("TBD")
    layer_dep_res.append("TBD")
    act_res.append("TBD")
    score_res.append(score_out)
    plt_teach_res.append(plt_teach_out)
    plt_pred_res.append(plt_best_out)

    save_the_log(path="drive/MyDrive/complex/complex_models",name=model_name[0])

Complex_2_LSTM_92_BERT_news

The model has 111,732,763 trainable parameters
The model has 3,881 trainable parameters
Epoch 1 		 Epoch time: 1m 44s
		 Training Loss: 0.6379210791122671 		 Validation Loss: 1.0227225973055913
		 Validation Loss Decreased(inf--->1.022723) 	 Saving The Model
Epoch 2 		 Epoch time: 1m 45s
		 Training Loss: 0.4970067121038163 		 Validation Loss: 1.1710784343572764
Epoch 3 		 Epoch time: 1m 45s
		 Training Loss: 0.3861650169537579 		 Validation Loss: 1.2985263191736662
Epoch 4 		 Epoch time: 1m 45s
		 Training Loss: 0.3071261369119826 		 Validation Loss: 1.370970904827118
Epoch 5 		 Epoch time: 1m 45s
		 Training Loss: 0.25660994124120556 		 Validation Loss: 1.3741920773799603
Epoch 6 		 Epoch time: 1m 45s
		 Training Loss: 0.22530225961393602 		 Validation Loss: 1.3236134006426885
Epoch 7 		 Epoch time: 1m 45s
		 Training Loss: 0.20303016635463447 		 Validation Loss: 1.245961537727943
Epoch 8 		 Epoch time: 1m 45s
		 Training Loss: 0.1872267334822666 		 Valid

In [ ]:
# train all complex with BERT -> need to rerun the preprocess -> there the check may be not epoch to epoch
complex_name_tmp = ["Complex_1", "Complex_2", "Complex_3"]
nums_name_tmp = ["Linear_108", "LSTM_92", "LSTM_36"]
news_name_tmp = ["BERT_news"]

model_name_tmp = []
model_num = []
model_complex = []

model_news = BERT_news()
model_news.load_state_dict(torch.load(f'drive/MyDrive/complex/news_models/71_model_BERT.pt'))

for complex_name_i in complex_name_tmp:
    for nums_name_i in nums_name_tmp:
        model_name_tmp.append(f"{complex_name_i}_{nums_name_i}_{news_name_tmp[0]}")

        if nums_name_i == "Linear_108":
          model_tmp = Linear_108_num()
          model_tmp.load_state_dict(torch.load('drive/MyDrive/complex/numerical_models/best_model_Linear_108.pt'))
          model_num.append(model_tmp)
        elif nums_name_i == "LSTM_92":
          model_tmp = LSTM_92_num()
          model_tmp.load_state_dict(torch.load('drive/MyDrive/complex/numerical_models/best_model_LSTM_92_num.pt'))
          model_num.append(model_tmp)
        elif nums_name_i == "LSTM_36":
          model_tmp = LSTM_36_num()
          model_tmp.load_state_dict(torch.load('drive/MyDrive/complex/numerical_models/best_model_LSTM_36_num.pt'))
          model_num.append(model_tmp)        
        else:
          pass

        if complex_name_i == "Complex_1":
          model_tmp_complex = Complex_1(model_tmp, model_news)
          model_complex.append(model_tmp_complex)
        elif complex_name_i == "Complex_2":
          model_tmp_complex = Complex_2(model_tmp, model_news)
          model_complex.append(model_tmp_complex)          
        elif complex_name_i == "Complex_3":
          model_tmp_complex = Complex_3(model_tmp, model_news)
          model_complex.append(model_tmp_complex)
        else:
          pass   



for i in range(7, len(model_name_tmp)):
    model_name = []
    model_name.append(model_name_tmp[i])

    model = model_complex[i]

    print(f"{model_name[0]}\n")

    # freeze the numerical and news weights
    print(f'The model has {count_parameters(model):,} trainable parameters')

    for p in model.model_num.parameters():
        p.requires_grad = False
    for p in model.model_news.parameters():
        p.requires_grad = False
    print(f'The model has {count_parameters(model):,} trainable parameters')

    optimizer = optim.Adam(model.parameters(), 0.0001)
    criterion = nn.MSELoss()

    e_res = []
    acc_res = []
    batch_s_res = []
    optimizer_start_res = []
    optimizer_end_res = []
    dropout_res = []
    layer_dep_res = []
    act_res = []
    score_res = []
    plt_teach_res = []
    plt_pred_res = []

    epoch_out, score_out, acc_out, plt_teach_out, plt_best_out = run_model_complex(batch_size_in=32,model_in=model,
              optimizer_in=optimizer,criterion_in=criterion,model_name_in=model_name[0])

    e_res.append(epoch_out)
    acc_res.append(acc_out)
    batch_s_res.append("32")
    optimizer_start_res.append("0.0001")
    optimizer_end_res.append("0.0001")
    dropout_res.append("TBD")
    layer_dep_res.append("TBD")
    act_res.append("TBD")
    score_res.append(score_out)
    plt_teach_res.append(plt_teach_out)
    plt_pred_res.append(plt_best_out)

    save_the_log(path="drive/MyDrive/complex/complex_models",name=model_name[0])

Complex_3_LSTM_92_BERT_news

The model has 111,790,155 trainable parameters
The model has 61,273 trainable parameters
Epoch 1 		 Epoch time: 1m 59s
		 Training Loss: 0.5876555943836432 		 Validation Loss: 1.260966149660257
		 Validation Loss Decreased(inf--->1.260966) 	 Saving The Model
Epoch 2 		 Epoch time: 1m 59s
		 Training Loss: 0.5203756654408533 		 Validation Loss: 1.3612547287574182
Epoch 3 		 Epoch time: 1m 59s
		 Training Loss: 0.43242618547928696 		 Validation Loss: 1.4385899213644175
Epoch 4 		 Epoch time: 1m 59s
		 Training Loss: 0.3103264354647616 		 Validation Loss: 1.3582571561519916
Epoch 5 		 Epoch time: 1m 59s
		 Training Loss: 0.1851999313394363 		 Validation Loss: 1.1415072954618013
		 Validation Loss Decreased(1.260966--->1.141507) 	 Saving The Model
Epoch 6 		 Epoch time: 1m 59s
		 Training Loss: 0.09761459409334772 		 Validation Loss: 0.938871661057839
		 Validation Loss Decreased(1.141507--->0.938872) 	 Saving The Model
Epoch 7 		 Epoch time: 1m 59s
		 Training

In [ ]:
# train all complex with BERT -> need to rerun the preprocess -> there the check may be not epoch to epoch
complex_name_tmp = ["Complex_1", "Complex_2", "Complex_3"]
nums_name_tmp = ["Linear_108", "LSTM_92", "LSTM_36"]
news_name_tmp = ["BERT_news"]

model_name_tmp = []
model_num = []
model_complex = []

model_news = BERT_news()
model_news.load_state_dict(torch.load(f'drive/MyDrive/complex/news_models/71_model_BERT.pt',map_location=torch.device('cpu')))

for complex_name_i in complex_name_tmp:
    for nums_name_i in nums_name_tmp:
        model_name_tmp.append(f"{complex_name_i}_{nums_name_i}_{news_name_tmp[0]}")

        if nums_name_i == "Linear_108":
          model_tmp = Linear_108_num()
          model_tmp.load_state_dict(torch.load('drive/MyDrive/complex/numerical_models/best_model_Linear_108.pt',map_location=torch.device('cpu')))
          model_num.append(model_tmp)
        elif nums_name_i == "LSTM_92":
          model_tmp = LSTM_92_num()
          model_tmp.load_state_dict(torch.load('drive/MyDrive/complex/numerical_models/best_model_LSTM_92_num.pt',map_location=torch.device('cpu')))
          model_num.append(model_tmp)
        elif nums_name_i == "LSTM_36":
          model_tmp = LSTM_36_num()
          model_tmp.load_state_dict(torch.load('drive/MyDrive/complex/numerical_models/best_model_LSTM_36_num.pt',map_location=torch.device('cpu')))
          model_num.append(model_tmp)        
        else:
          pass

        if complex_name_i == "Complex_1":
          model_tmp_complex = Complex_1(model_tmp, model_news)
          model_complex.append(model_tmp_complex)
        elif complex_name_i == "Complex_2":
          model_tmp_complex = Complex_2(model_tmp, model_news)
          model_complex.append(model_tmp_complex)          
        elif complex_name_i == "Complex_3":
          model_tmp_complex = Complex_3(model_tmp, model_news)
          model_complex.append(model_tmp_complex)
        else:
          pass   



for i in [8]:
    model_name = []
    model_name.append(model_name_tmp[i])

    model = model_complex[i]

    print(f"{model_name[0]}\n")

    model.load_state_dict(torch.load('drive/MyDrive/complex/complex_models/best_model_' + str(model_name[0]) + '.pt',map_location=torch.device('cpu')))

    predict_test_array = []

    with torch.no_grad():
        for i in range(len(test_X_num)):
            model.eval()
            predict = model(test_X_num[i].reshape(1,-1).to(device), test_X_news[i].reshape(1,-1).to(device))        
            predict_test_array.append(predict.item()*output_df.std()+output_df.mean()) 

    real_test_array = []
    real_trend_array = test["Trend target"]  
    for y in test_Y_num:
      real_test_array.append(y*output_df.std()+output_df.mean())


    # calculate score
    criterion = nn.MSELoss()
    score = calc_score(predict_test_array,real_test_array)    
    print(f"\nScore: {score}\n")      

    #check the trend -> create an array with the real and with the predicted trend
    #if the current value is bigger than before -> 1
    #otherwise (same or smaller) -> 0
    predicted_trend_array = []
    for element in range(len(predict_test_array)):
        real_today_close = test["Close"][element]*input_df["Close"].std()+input_df["Close"].mean()
        if real_today_close > predict_test_array[element].values:
            predicted_trend_array.append(0)
        else:
            predicted_trend_array.append(1)

    #check the number of differences
    trend_diff_array = []
    for element in range(len(real_trend_array)):
        if real_trend_array[element] != predicted_trend_array[element]:
          trend_diff_array.append(element)

    #percentage of good predict
    acc = (len(real_trend_array)-len(trend_diff_array))/len(real_trend_array)*100
    print(f"Accuracy: {acc}\n")  
    #visualize it
    f2 = plt.figure(figsize=(24,12))
    plt.title("Predicted and real close prices", fontsize = 18)
    plt.plot(real_test_array, color = "blue", linewidth = 4,
            label = "Real")
    plt.plot(predict_test_array, color = "red", linewidth = 2,
            label = "Predicted")
    plt.xlabel("Date",fontsize = 18)
    plt.legend(fontsize = 18)
    f2.set_size_inches(12,6)
    plt_best = f2
    plt.close()

Complex_3_LSTM_36_BERT_news


Score: 8443.171319796955

Accuracy: 48.984771573604064

